In [1]:
import os
import sys
import time

import function_board as fb
import function_tool as ft
import original_code.function_get_aiming_grid as fgag
import original_code.function_evaluate_policy as fep

import numpy as np
np.set_printoptions(precision=4)
np.set_printoptions(linewidth=300)
np.set_printoptions(threshold=300)

import torch
torch.set_printoptions(precision=4)
torch.set_printoptions(linewidth=300)
torch.set_printoptions(threshold=300)

## Noturn Reference

In [2]:
data_parameter_dir = fb.data_parameter_dir
result_dir = './result'  
name_pa = 'player{}'.format(10)  

[aiming_grid, prob_grid_normalscore_nt, prob_grid_singlescore_nt, prob_grid_doublescore_nt, prob_grid_triplescore_nt, prob_grid_bullscore_nt] = fgag.load_aiming_grid(name_pa, data_parameter_dir=data_parameter_dir, grid_version='custom_no_tokens')
[aiming_grid, prob_grid_normalscore_t, prob_grid_singlescore_t, prob_grid_doublescore_t, prob_grid_triplescore_t, prob_grid_bullscore_t] = fgag.load_aiming_grid(name_pa, data_parameter_dir=data_parameter_dir, grid_version='custom_tokens')

num_aiming_location = aiming_grid.shape[0]
prob_normalscore_1tosmax_dic = {}
prob_normalscore_1tosmaxsum_dic = {}
prob_normalscore_1tosmax_dic_t = {}
prob_normalscore_1tosmaxsum_dic_t = {}
prob_grid_doublescore_dic_t = None
prob_grid_doublescore_dic= None

# Tokens
for score_max in range(0,61):
    score_max_plus1 = score_max + 1 
    prob_normalscore_1tosmax_dic_t[score_max] = np.array(prob_grid_normalscore_t[:,1:score_max_plus1])
    prob_normalscore_1tosmaxsum_dic_t[score_max] = prob_normalscore_1tosmax_dic_t[score_max].sum(axis=1)
if prob_grid_doublescore_dic_t is None:
    prob_doublescore_dic_t = {}
    for doublescore_index in range(20):
        doublescore = 2*(doublescore_index+1)
        prob_doublescore_dic_t[doublescore] = np.array(prob_grid_doublescore_t[:,doublescore_index])
else:
    prob_doublescore_dic_t = prob_grid_doublescore_dic_t
prob_DB_t = np.array(prob_grid_bullscore_t[:,1])

# No Tokens
for score_max in range(0,61):
    score_max_plus1 = score_max + 1 
    prob_normalscore_1tosmax_dic[score_max] = np.array(prob_grid_normalscore[:,1:score_max_plus1])
    prob_normalscore_1tosmaxsum_dic[score_max] = prob_normalscore_1tosmax_dic[score_max].sum(axis=1)
if prob_grid_doublescore_dic is None:
    prob_doublescore_dic = {}
    for doublescore_index in range(20):
        doublescore = 2*(doublescore_index+1)
        prob_doublescore_dic[doublescore] = np.array(prob_grid_doublescore[:,doublescore_index])
else:
    prob_doublescore_dic = prob_grid_doublescore_dic
prob_DB = np.array(prob_grid_bullscore[:,1])


load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_no_tokens/player10_gaussin_prob_grid_custom_no_tokens.pkl
load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_tokens/player10_gaussin_prob_grid_custom_tokens.pkl


NameError: name 'prob_grid_normalscore' is not defined

In [3]:
## How did probabilities work in the no-turn case? ## 

# Probabilities of all scores from throw actions are zero when using the token probability dictionary 
prob_normalscore_1tosmax_dic_t[60].sum(axis=1)[:740]

# Probabilities of all scores from token actions are zero when using the no-token probability dictionary 
prob_normalscore_1tosmax_dic[60].sum(axis=1)[:740]

# Therefore, they are mutually exclusive

# Cumulative probability of each action across all scores should be one if there are no restrictions on max score

# Max score is a shortcut that allows for easier computation at the other end

# In the noturn case everything is simplified because if you go bust or miss your score stays the same. 
# In the turn case we will need to split this out a bit more 

KeyError: 60

## Turn Exploration

In [4]:
## single player game without the turn feature
def solve_dp_noturn(aiming_grid, prob_grid_normalscore, prob_grid_doublescore=None, prob_grid_bullscore=None, prob_grid_doublescore_dic=None):
    """
    Solve the single player game without the turn feature. Find the optimal policy to minimize the expected number of throws for reaching zero score. 
    Args: 
        the action set and the hitting probability associated with the skill model
    
    Returns: 
        optimal_value[score_state]: the expected number of throws for reaching zero from score_state=2,...,501.
        optimal_action_index[score_state]: the index of the aiming location used for score_state=2,...,501.
    """

    num_aiming_location = aiming_grid.shape[0]
    prob_normalscore_1tosmax_dic = {}
    prob_normalscore_1tosmaxsum_dic = {}
    for score_max in range(0,61):
        score_max_plus1 = score_max + 1 
        prob_normalscore_1tosmax_dic[score_max] = np.array(prob_grid_normalscore[:,1:score_max_plus1])
        prob_normalscore_1tosmaxsum_dic[score_max] = prob_normalscore_1tosmax_dic[score_max].sum(axis=1)
    if prob_grid_doublescore_dic is None:
        prob_doublescore_dic = {}
        for doublescore_index in range(20):
            doublescore = 2*(doublescore_index+1)
            prob_doublescore_dic[doublescore] = np.array(prob_grid_doublescore[:,doublescore_index])
    else:
        prob_doublescore_dic = prob_grid_doublescore_dic
    prob_DB = np.array(prob_grid_bullscore[:,1])

    ## possible state: s = 0,1(not possible),2,...,501
    optimal_value = np.zeros(502)
    #optimal_value[1] = np.nan
    optimal_action_index = np.zeros(502, np.int32)
    optimal_action_index[0] = -1
    optimal_action_index[1] = -1
        
    for score_state in range(2,502):            
        ## use matrix operation to search all aiming locations
        
        ## transit to less score state    
        ## s1 = min(score_state-2, 60)
        ## p[z=1]*v[score_state-1] + p[z=2]*v[score_state-2] + ... + p[z=s1]*v[score_state-s1]
        score_max = min(score_state-2, 60)
        score_max_plus1 = score_max + 1 
        ## transit to next state
        num_tothrow = 1.0 + prob_normalscore_1tosmax_dic[score_max].dot(optimal_value[score_state-1:score_state-score_max-1:-1])
        ## probability of transition to state other than s itself
        prob_otherstate = prob_normalscore_1tosmaxsum_dic[score_max]
        
        ## transit to the end of game
        if (score_state == fb.score_DB): ## hit double bull
            prob_otherstate += prob_DB
        elif (score_state <= 40 and score_state%2==0): ## hit double
            prob_otherstate += prob_doublescore_dic[score_state]
        else: ## game does not end
            pass
        
        ## expected number of throw for all aiming locations
        prob_otherstate = np.maximum(prob_otherstate, 0)
        num_tothrow = num_tothrow / prob_otherstate
                            
        ## searching
        optimal_value[score_state] = num_tothrow.min()
        optimal_action_index[score_state] = num_tothrow.argmin()

    return [optimal_value, optimal_action_index]


In [5]:
def init_probabilities( aiming_grid, prob_grid_normalscore_nt, prob_grid_doublescore_nt, prob_grid_bullscore_nt, prob_grid_normalscore_t, prob_grid_doublescore_t, prob_grid_bullscore_t):
    
    # aiming grid
    num_aiming_location = aiming_grid.shape[0]
    prob_normalscore_nt = prob_grid_normalscore_nt
    prob_normalscore_t = prob_grid_normalscore_t
    prob_doublescore_dic_nt = {}
    prob_doublescore_dic_t = {}
    for doublescore_index in range(20):
        doublescore = 2*(doublescore_index+1)
        prob_doublescore_dic_nt[doublescore] = np.array(prob_grid_doublescore_nt[:,doublescore_index])
        prob_doublescore_dic_t[doublescore] = np.array(prob_grid_doublescore_t[:,doublescore_index])
    prob_DB_nt = np.array(prob_grid_bullscore_nt[:,1])
    prob_DB_t = np.array(prob_grid_bullscore_t[:,1])

    ## the probability of not bust for each action given score_max=i (score_remain=i+2)
    prob_bust_dic_nt = {}
    prob_notbust_dic_nt = {}
    prob_bust_dic_t = {}
    prob_notbust_dic_t = {}
    for score_max in range(60):    
        ## transit to next throw or turn
        prob_notbust_nt = prob_grid_normalscore_nt[:,0:score_max+1].sum(axis=1)
        prob_notbust_t = prob_grid_normalscore_t[:,0:score_max+1].sum(axis=1)
        ## transit to the end of game
        score_remain = score_max + 2
        if (score_remain == fb.score_DB):
            prob_notbust_nt += prob_DB_nt
            prob_notbust_t += prob_DB_t
        elif (score_remain <= 40 and score_remain%2==0):
            prob_notbust_nt += prob_doublescore_dic_nt[score_remain]
            prob_notbust_t += prob_doublescore_dic_t[score_remain]
        ##
        prob_notbust_nt = np.minimum(np.maximum(prob_notbust_nt, 0),1)
        prob_notbust_dic_nt[score_max] = prob_notbust_nt
        prob_bust_dic_nt[score_max] = 1 - prob_notbust_dic_nt[score_max]
        prob_notbust_t = np.minimum(np.maximum(prob_notbust_t, 0),1)
        prob_notbust_dic_t[score_max] = prob_notbust_t
        prob_bust_dic_t[score_max] = 1 - prob_notbust_dic_t[score_max]
    
    return num_aiming_location, prob_normalscore_nt, prob_doublescore_dic_nt, prob_DB_nt, prob_bust_dic_nt, prob_notbust_dic_nt, prob_normalscore_t, prob_doublescore_dic_t, prob_DB_t, prob_bust_dic_t, prob_notbust_dic_t

In [368]:
# If you have a max score of 60 you cannot go bust as this is the max throw score for a single throw.
# The bust dictionary contains the probability of going bust given each action. 
# Key is the max score, index of array is action index, value is probability of going bust

prob_bust_dic = {}
for k in prob_bust_dic_t.keys():
    prob_bust_dic[k] = prob_bust_dic_t[k] + prob_bust_dic_nt[k]

prob_bust_dic

{0: array([1., 1., 1., ..., 1., 1., 1.]),
 1: array([0.9611, 0.7815, 0.7229, ..., 1.    , 1.    , 1.    ]),
 2: array([0.9611, 0.7815, 0.7229, ..., 1.    , 1.    , 1.    ]),
 3: array([0.9577, 0.7815, 0.7229, ..., 1.    , 1.    , 1.    ]),
 4: array([0.9577, 0.7815, 0.7228, ..., 1.    , 1.    , 1.    ]),
 5: array([0.8981, 0.5099, 0.4768, ..., 1.    , 1.    , 1.    ]),
 6: array([0.8981, 0.5099, 0.4768, ..., 1.    , 1.    , 1.    ]),
 7: array([0.8981, 0.5099, 0.4768, ..., 1.    , 1.    , 1.    ]),
 8: array([0.8981, 0.5099, 0.4768, ..., 1.    , 1.    , 1.    ]),
 9: array([0.8981, 0.5099, 0.476 , ..., 1.    , 1.    , 1.    ]),
 10: array([0.8981, 0.5099, 0.476 , ..., 1.    , 1.    , 1.    ]),
 11: array([0.8981, 0.5099, 0.476 , ..., 1.    , 1.    , 1.    ]),
 12: array([0.898 , 0.5075, 0.443 , ..., 1.    , 1.    , 1.    ]),
 13: array([0.898 , 0.5075, 0.443 , ..., 1.    , 1.    , 1.    ]),
 14: array([0.898 , 0.5075, 0.4423, ..., 1.    , 1.    , 1.    ]),
 15: array([0.898 , 0.5075, 0

In [6]:
def init_data_structures():
    optimal_value_rt3 = np.zeros(502) #vector: optimal value for the beginning state of each turn (rt=3)
    optimal_value_dic = {} ## first key: score=0,2,...,501, second key: remaining throws=3,2,1
    optimal_action_index_dic = {}
    num_iteration_record = np.zeros(502, dtype=np.int32)
    
    state_len_vector = np.zeros(4, dtype=np.int32)
    state_value  = [None]  ## optimal value (expected # of turns to finish the game) for each state in the current playing turn
    state_action = [None]  ## aimming locations for for each state in the current playing turn
    action_diff  = [None]
    value_relerror = np.zeros(4)
    for rt in [1,2,3]:
        ## for rt=3: possible score_gained = 0
        ## for rt=2: possible score_gained = 0,1,...,60
        ## for rt=1: possible score_gained = 0,1,...,120
        this_throw_state_len = fb.maxhitscore*(3-rt) + 1
        state_value.append(np.ones(this_throw_state_len)*fb.largenumber)
        state_action.append(np.ones(this_throw_state_len, np.int32)*fb.infeasible_marker)
        action_diff.append(np.ones(this_throw_state_len))
    state_value_update = ft.copy_numberarray_container(state_value)
    state_action_update = ft.copy_numberarray_container(state_action)

    return optimal_value_rt3, optimal_value_dic, optimal_action_index_dic, num_iteration_record, state_len_vector, state_value, state_action, action_diff, value_relerror, state_value_update, state_action_update

In [427]:
def get_noturn_policy_tokens(score,tokens,no_token_bool=False):

    tokens_plus_one = tokens + 1

    [aiming_grid, prob_grid_normalscore_nt, prob_grid_singlescore_nt, prob_grid_doublescore_nt, prob_grid_triplescore_nt, prob_grid_bullscore_nt] = fgag.load_aiming_grid(name_pa, data_parameter_dir=data_parameter_dir, grid_version='custom_no_tokens')
    [aiming_grid, prob_grid_normalscore_t, prob_grid_singlescore_t, prob_grid_doublescore_t, prob_grid_triplescore_t, prob_grid_bullscore_t] = fgag.load_aiming_grid(name_pa, data_parameter_dir=data_parameter_dir, grid_version='custom_tokens')

    optimal_action_index_dic = {}

    state_len_vector = np.zeros(4, dtype=np.int32)
    state_value  = [None]  ## optimal value (expected # of turns to finish the game) for each state in the current playing turn
    state_action = [None]  ## aimming locations for for each state in the current playing turn
    action_diff  = [None]

    for rt in [1,2,3]:
        ## for rt=3: possible score_gained = 0
        ## for rt=2: possible score_gained = 0,1,...,60
        ## for rt=1: possible score_gained = 0,1,...,120
        this_throw_state_len = fb.maxhitscore*(3-rt) + 1
        state_value.append(np.ones((tokens_plus_one,this_throw_state_len))*fb.largenumber)
        state_action.append(np.ones((tokens_plus_one,this_throw_state_len), np.int32)*fb.infeasible_marker)
        action_diff.append(np.ones((tokens_plus_one,this_throw_state_len)))

    state_action_update = ft.copy_numberarray_container(state_action)

    ## use no_turn policy as the initial policy
    #[noturn_optimal_value, noturn_optimal_action_index] = solve_dp_noturn(aiming_grid, prob_grid_normalscore, prob_grid_doublescore, prob_grid_bullscore)
    [noturn_optimal_value, noturn_optimal_action_index] = h.solve_dp_noturn_tokens(aiming_grid, prob_grid_normalscore_nt, prob_grid_normalscore_t, tokens = tokens, prob_grid_doublescore=prob_grid_doublescore_nt, prob_grid_bullscore=prob_grid_bullscore_nt, prob_grid_doublescore_dic=None, prob_grid_doublescore_t=prob_grid_doublescore_t, prob_grid_bullscore_t=prob_grid_bullscore_t, prob_grid_doublescore_dic_t=None)

    t1 = time.time()
    for score_state in range(2,score+1):

        ## initialization 
        for rt in [1,2,3]:

            for tok in range(tokens_plus_one):
            
                ## for rt=3: score_gained = 0
                ## for rt=2: score_gained = 0,1,...,min(s-2,60)
                ## for rt=1: score_gained = 0,1,...,min(s-2,120)
                this_throw_state_len = min(score_state-2, fb.maxhitscore*(3-rt)) + 1
                state_len_vector[rt] = this_throw_state_len
                        
                ## initialize the starting policy: 
                ## use no_turn action in (s, i, u=0)
                ## use turn action (s-1, i, u-1) in (s, i, u!=0) if (s-1, i, u-1) is feasible state
                state_action[rt][tok][0] = noturn_optimal_action_index[tok][score_state]      
                        
                for score_gained in range(1,this_throw_state_len):                
                    if fb.state_feasible_array[rt, score_gained]:  ## if True
                        if fb.state_feasible_array[rt, score_gained-1]:
                            state_action[rt][tok][score_gained] = optimal_action_index_dic[score_state-1][rt][tok][score_gained-1]
                        else:                        
                            state_action[rt][tok][score_gained] = noturn_optimal_action_index[tok][score_state-score_gained]
                    else:
                        state_action[rt][tok][score_gained] = fb.infeasible_marker
        
        for rt in [1,2,3]:
            state_action_update[rt][tok][fb.state_infeasible[rt]] = fb.infeasible_marker
        
        optimal_action_index_dic[score_state] = ft.copy_numberarray_container(state_action_update)
   
    if no_token_bool:
        for i in range(1,len(state_action)):
            state_action[i] = state_action[i][0]

    return state_action

In [ ]:
num_aiming_location, prob_normalscore_nt, prob_doublescore_dic_nt, prob_DB_nt, prob_bust_dic_nt, prob_notbust_dic_nt, prob_normalscore_t, prob_doublescore_dic_t, prob_DB_t, prob_bust_dic_t, prob_notbust_dic_t = init_probabilities( aiming_grid, prob_grid_normalscore_nt, prob_grid_doublescore_nt, prob_grid_bullscore_nt, prob_grid_normalscore_t, prob_grid_doublescore_t, prob_grid_bullscore_t)    


In [1253]:
score = 501

actions_their_format = get_noturn_policy_tokens(score,0,no_token_bool=True)
actions_our_format = get_noturn_policy_tokens(score,0,no_token_bool=False)

num_aiming_location, prob_normalscore_nt, prob_doublescore_dic_nt, prob_DB_nt, prob_bust_dic_nt, prob_notbust_dic_nt, prob_normalscore_t, prob_doublescore_dic_t, prob_DB_t, prob_bust_dic_t, prob_notbust_dic_t = init_probabilities( aiming_grid, prob_grid_normalscore_nt, prob_grid_doublescore_nt, prob_grid_bullscore_nt, prob_grid_normalscore_t, prob_grid_doublescore_t, prob_grid_bullscore_t)    

their_probs = fep.solve_turn_transit_probability_fast(262,actions_their_format,prob_grid_normalscore_nt,prob_grid_doublescore_nt,prob_grid_bullscore_nt,prob_bust_dic_nt)
our_probs = solve_turn_transit_probability_fast_token(262, actions_our_format, 0, prob_grid_normalscore_nt, prob_grid_doublescore_nt, prob_grid_bullscore_nt, prob_bust_dic_nt,prob_grid_normalscore_t, prob_grid_doublescore_t, prob_grid_bullscore_t, prob_bust_dic_t)

print('theirs:',their_probs['score'].sum())
print('ours:',our_probs['score'].sum())

load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_no_tokens/player10_gaussin_prob_grid_custom_no_tokens.pkl
load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_tokens/player10_gaussin_prob_grid_custom_tokens.pkl
load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_no_tokens/player10_gaussin_prob_grid_custom_no_tokens.pkl
load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_tokens/player10_gaussin_prob_grid_custom_tokens.pkl
theirs: 1.0
ours: 0.9999999999999988


/Users/rachael/Desktop/darts-thesis/helpers.py:506: RuntimeWarning: divide by zero encountered in divide
  num_tothrow = num_tothrow / prob_otherstate
/Users/rachael/Desktop/darts-thesis/helpers.py:506: RuntimeWarning: overflow encountered in divide
  num_tothrow = num_tothrow / prob_otherstate


In [1254]:
prob_grid_normalscore_nt

array([[1.2150e-05, 2.2207e-01, 2.8532e-05, ..., 0.0000e+00, 0.0000e+00, 3.4711e-02],
       [2.7856e-07, 2.5306e-01, 1.1642e-06, ..., 0.0000e+00, 0.0000e+00, 3.0777e-03],
       [7.0439e-09, 2.1636e-01, 8.7676e-05, ..., 0.0000e+00, 0.0000e+00, 8.0084e-05],
       ...,
       [1.0000e+00, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00, 0.0000e+00],
       [1.0000e+00, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00, 0.0000e+00],
       [1.0000e+00, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00, 0.0000e+00]])

In [505]:
violators = []
for i in range(2,502):
    for t in range(10):
        p = 0 
        actions_our_format = get_noturn_policy_tokens(i,t,no_token_bool=False)
        our_probs = solve_turn_transit_probability_fast_token(i, actions_our_format, t, prob_grid_normalscore_nt, prob_grid_doublescore_nt, prob_grid_bullscore_nt, prob_bust_dic_nt,prob_grid_normalscore_t, prob_grid_doublescore_t, prob_grid_bullscore_t, prob_bust_dic_t)
        p += our_probs['score'].sum()
        p += our_probs['finish']
        p += our_probs['bust'].sum()
        if p > 1: 
            violators.append((i,t,p))


load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_no_tokens/player10_gaussin_prob_grid_custom_no_tokens.pkl
load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_tokens/player10_gaussin_prob_grid_custom_tokens.pkl
load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_no_tokens/player10_gaussin_prob_grid_custom_no_tokens.pkl
load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_tokens/player10_gaussin_prob_grid_custom_tokens.pkl
load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_no_tokens/player10_gaussin_prob_grid_custom_no_tokens.pkl
load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_tokens/player10_gaussin_prob_grid_custom_tokens.pkl


/Users/rachael/Desktop/darts-thesis/helpers.py:506: RuntimeWarning: divide by zero encountered in divide
  num_tothrow = num_tothrow / prob_otherstate
/Users/rachael/Desktop/darts-thesis/helpers.py:506: RuntimeWarning: overflow encountered in divide
  num_tothrow = num_tothrow / prob_otherstate
/Users/rachael/Desktop/darts-thesis/helpers.py:544: RuntimeWarning: divide by zero encountered in divide
  num_tothrow = num_tothrow / prob_otherstate
/Users/rachael/Desktop/darts-thesis/helpers.py:544: RuntimeWarning: overflow encountered in divide
  num_tothrow = num_tothrow / prob_otherstate


load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_no_tokens/player10_gaussin_prob_grid_custom_no_tokens.pkl
load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_tokens/player10_gaussin_prob_grid_custom_tokens.pkl
load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_no_tokens/player10_gaussin_prob_grid_custom_no_tokens.pkl
load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_tokens/player10_gaussin_prob_grid_custom_tokens.pkl
load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_no_tokens/player10_gaussin_prob_grid_custom_no_tokens.pkl
load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_tokens/player10_gaussin_prob_grid_custom_tokens.pkl
load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_no_tokens/player10_gaussin_prob_grid_custom_no_tokens.pkl
load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_tokens/player10_gaussin_prob_grid_custom_tokens.pkl
load_pickle from ./data_parameter/player

In [508]:
counter = 0 
for v in violators: 
    if violators[i][2] > 1.0000000001: 
        counter+=1
        print(v)

print(counter)


(16, 0, 1.0000000000000002)
(19, 1, 1.0000000000000002)
(21, 1, 1.0000000000000002)
(23, 1, 1.0000000000000002)
(24, 0, 1.0000000000000002)
(25, 1, 1.0000000000000002)
(27, 1, 1.0000000000000002)
(29, 1, 1.0000000000000002)
(31, 1, 1.0000000000000002)
(33, 1, 1.0000000000000002)
(35, 1, 1.0000000000000002)
(37, 1, 1.0000000000000002)
(39, 1, 1.0000000000000002)
(41, 0, 1.0000017865891258)
(41, 1, 1.0000000000000002)
(42, 1, 1.0000087792606411)
(43, 1, 1.0000000000000002)
(44, 1, 1.0000087792606411)
(45, 0, 1.0000000000000002)
(46, 1, 1.0000087792606411)
(49, 0, 1.0000089575341402)
(50, 0, 1.000008678727265)
(53, 1, 1.0000000000000002)
(55, 0, 1.0004654957028725)
(56, 0, 1.000076124274205)
(57, 0, 1.0000000000000002)
(59, 0, 1.001407287096259)
(61, 0, 1.001407287096259)
(62, 0, 1.000905633663316)
(63, 0, 1.000843999580522)
(64, 0, 1.0009056336633164)
(65, 0, 1.0000089578318867)
(66, 1, 1.0000000000000007)
(67, 0, 1.001407287096259)
(67, 1, 1.0004654965759607)
(68, 0, 1.0009056336633166)

In [834]:
prob_grid_bullscore_comb = np.zeros(prob_grid_bullscore_t.shape)
prob_grid_bullscore_comb[:729,:] = prob_grid_bullscore_nt[:729,:]
prob_grid_bullscore_comb[729:] = prob_grid_bullscore_t[729:]
prob_grid_bullscore_comb.shape



array([6.8299e-23, 3.8593e-32, 1.0072e-39, ..., 0.0000e+00, 0.0000e+00, 0.0000e+00])

In [1123]:
import helpers as h
tokens = 10
tokens_plus_one = tokens + 1

## Initialize probability objects
num_aiming_location, prob_normalscore_nt, prob_doublescore_dic_nt, prob_DB_nt, prob_bust_dic_nt, prob_notbust_dic_nt, prob_normalscore_t, prob_doublescore_dic_t, prob_DB_t, prob_bust_dic_t, prob_notbust_dic_t = init_probabilities( aiming_grid, prob_grid_normalscore_nt, prob_grid_doublescore_nt, prob_grid_bullscore_nt, prob_grid_normalscore_t, prob_grid_doublescore_t, prob_grid_bullscore_t)    

# Initialize joint probability notbust dictionary 
prob_notbust_dic = {}
for score_max in prob_notbust_dic_t.keys():
    prob_notbust_dic[score_max] = np.zeros(prob_notbust_dic_t[0].shape[0]) # 791 
    prob_notbust_dic[score_max][:729] += prob_notbust_dic_nt[score_max][:729] 
    prob_notbust_dic[score_max][729:] += prob_notbust_dic_t[score_max][729:]

# Intialize joint probability bust dictionary 
prob_bust_dic = {}
for score_max in prob_notbust_dic_t.keys():
    prob_bust_dic[score_max] = np.zeros(prob_bust_dic_t[0].shape[0]) # 791 
    prob_bust_dic[score_max][:729] += prob_bust_dic_nt[score_max][:729] 
    prob_bust_dic[score_max][729:] += prob_bust_dic_t[score_max][729:]


prob_normalscore_tensor_nt = torch.from_numpy(prob_normalscore_nt)
prob_normalscore_tensor_t = torch.from_numpy(prob_normalscore_t)

prob_normalscore_tensor_nt = torch.from_numpy(prob_normalscore_nt)
prob_normalscore_tensor_t = torch.from_numpy(prob_normalscore_t)
# prob_doublescore_dic = prob_doublescore_dic_nt
# prob_DB = prob_DB_nt
# prob_bust_dic = prob_bust_dic_nt
# prob_notbust_dic = prob_notbust_dic_nt

iteration_round_limit = 20

optimal_value_rt3 = np.zeros((tokens_plus_one,502)) #vector: optimal value for the beginning state of each turn (rt=3)
optimal_value_dic = {} ## first key: score=0,2,...,501, second key: remaining throws=3,2,1
optimal_action_index_dic = {}
num_iteration_record = np.zeros(502, dtype=np.int32)

state_len_vector = np.zeros(4, dtype=np.int32)
state_value  = [None]  ## optimal value (expected # of turns to finish the game) for each state in the current playing turn
state_action = [None]  ## aimming locations for for each state in the current playing turn
action_diff  = [None]
value_relerror = np.zeros(4)

for rt in [1,2,3]:
    ## for rt=3: possible score_gained = 0
    ## for rt=2: possible score_gained = 0,1,...,60
    ## for rt=1: possible score_gained = 0,1,...,120
    this_throw_state_len = fb.maxhitscore*(3-rt) + 1
    state_value.append(np.ones((tokens_plus_one,this_throw_state_len))*fb.largenumber)
    state_action.append(np.ones((tokens_plus_one,this_throw_state_len), np.int32)*fb.infeasible_marker)
    action_diff.append(np.ones((tokens_plus_one,this_throw_state_len)))
state_value_update = ft.copy_numberarray_container(state_value)
state_action_update = ft.copy_numberarray_container(state_action)

## use no_turn policy as the initial policy
#[noturn_optimal_value, noturn_optimal_action_index] = solve_dp_noturn(aiming_grid, prob_grid_normalscore, prob_grid_doublescore, prob_grid_bullscore)
[noturn_optimal_value, noturn_optimal_action_index] = h.solve_dp_noturn_tokens(aiming_grid, prob_grid_normalscore_nt, prob_grid_normalscore_t, tokens = tokens, prob_grid_doublescore=prob_grid_doublescore_nt, prob_grid_bullscore=prob_grid_bullscore_nt, prob_grid_doublescore_dic=None, prob_grid_doublescore_t=prob_grid_doublescore_t, prob_grid_bullscore_t=prob_grid_bullscore_t, prob_grid_doublescore_dic_t=None)

t1 = time.time()
for score_state in range(2,502):

    ## initialization 
    for rt in [1,2,3]:

        for tok in range(tokens_plus_one):
        
            ## for rt=3: score_gained = 0
            ## for rt=2: score_gained = 0,1,...,min(s-2,60)
            ## for rt=1: score_gained = 0,1,...,min(s-2,120)
            this_throw_state_len = min(score_state-2, fb.maxhitscore*(3-rt)) + 1
            state_len_vector[rt] = this_throw_state_len
                    
            ## initialize the starting policy: 
            ## use no_turn action in (s, i, u=0)
            ## use turn action (s-1, i, u-1) in (s, i, u!=0) if (s-1, i, u-1) is feasible state
            state_action[rt][tok][0] = noturn_optimal_action_index[tok][score_state]            
            for score_gained in range(1,this_throw_state_len):                
                if fb.state_feasible_array[rt, score_gained]:  ## if True
                    if fb.state_feasible_array[rt, score_gained-1]:
                        state_action[rt][tok][score_gained] = optimal_action_index_dic[score_state-1][rt][tok][score_gained-1]
                    else:                        
                        state_action[rt][tok][score_gained] = noturn_optimal_action_index[tok][score_state-score_gained]
                else:
                    state_action[rt][tok][score_gained] = fb.infeasible_marker
    
    for tok in range(tokens_plus_one):
        # policy iteration
        for round_index in range(iteration_round_limit):

            ## --------------------------------------------- ##
            ##                 POLICY EVALUATION
            ## --------------------------------------------- ##
            rt = 3
            score_gained = 0
            score_max_turn = min(score_state-2, 3*fb.maxhitscore)

            # Get the transit probabilities 
            prob_turn_transit = solve_turn_transit_probability_fast_token(score_state, state_action, tok, prob_grid_normalscore_nt, prob_grid_doublescore_nt, prob_grid_bullscore_nt, prob_bust_dic_nt,prob_grid_normalscore_t, prob_grid_doublescore_t, prob_grid_bullscore_t, prob_bust_dic_t)
            
            tokens_max_turn = min(prob_turn_transit['bust'].shape[0]-1,tok)

            # Make updates below 
            prob_turn_zeroscore = prob_turn_transit['bust'][0] + prob_turn_transit['score'][0,0]

            new_value_rt3 = 1 

            # Don't use a token 
            new_value_rt3 += np.dot(prob_turn_transit['score'][0,1:], optimal_value_rt3[tok,score_state-1:score_state-score_max_turn-1:-1])
            new_value_rt3
            # Use at least one token 
            if tok > 0: 
                # Go to subsequent score after using at least one token
                for i in range(tokens_max_turn+1):
                    new_value_rt3 += np.dot(np.flip(prob_turn_transit['score'][0:tokens_max_turn+1,:],axis=0)[i], optimal_value_rt3[tok-tokens_max_turn:tok+1,score_state:score_state-score_max_turn-1:-1][i])
                # Go bust after using at least one token 
                new_value_rt3 += np.dot(prob_turn_transit['bust'][1:tokens_max_turn+1] , optimal_value_rt3[tok:tok-tokens_max_turn:-1,score_state])

            # Normalize over probability of moving to a new state 
            new_value_rt3 = new_value_rt3 / (1-prob_turn_zeroscore)
            new_value_rt3

            state_value_update[rt][tok][score_gained] = new_value_rt3
            optimal_value_rt3[tok][score_state] = new_value_rt3
            #print('evaluate rt3 value= {}'.format(new_value_rt3)

            ## --------------------------------------------- ##
            ##                 POLICY IMPROVEMENT
            ## --------------------------------------------- ## 

            for rt in [1,2,3]: 

                this_throw_state_len = state_len_vector[rt]
                state_notbust_len =  max(min(score_state-61, this_throw_state_len),0)
                token_index = min(2,tok+1)

                ## CASE 1: state which can not bust.  score_state-score_gained>=62 
                if (state_notbust_len > 0):
                    # One throw remaining and first round of iteration 
                    if (rt==1 and round_index==0):
                        ## combine all non-bust states together 
                        state_notbust_update_index = state_notbust_len                    
                        next_state_value_array = np.zeros((token_index, 61, state_notbust_len))                    
                        for score_gained in range(state_notbust_len):
                            ## skip infeasible state
                            if not fb.state_feasible_array[rt, score_gained]:
                                continue
                            score_remain = score_state - score_gained
                            score_max = 60 ## always 60 here
                            score_max_plus1 = score_max + 1
                            if tok > 0: 
                                new_state_vals = optimal_value_rt3[tok-1:tok+1]
                                new_state_vals = np.flip(new_state_vals,axis=0)
                                next_state_value_array[:,:,score_gained] = new_state_vals[:,score_remain:score_remain-score_max_plus1:-1]
                            else: 
                                new_state_vals = optimal_value_rt3[:1]
                                next_state_value_array[:1,:,score_gained] = new_state_vals[:,score_remain:score_remain-score_max_plus1:-1]

                    # Two throws remaining and first round of iteration 
                    elif (rt==2 and (round_index==0 or score_state<182)):
                        ## combine all non-bust states together 
                        state_notbust_update_index = state_notbust_len
                        next_state_value_array = np.zeros((token_index, 61, state_notbust_len))                    
                        for score_gained in range(state_notbust_len):
                            ## skip infeasible state
                            if not fb.state_feasible_array[rt, score_gained]:
                                continue
                            score_remain = score_state - score_gained
                            score_max = 60 ## always 60 here
                            score_max_plus1 = score_max + 1
                            if tok > 0: 
                                new_state_vals = state_value_update[rt-1][tok-1:tok+1]
                                new_state_vals = np.flip(new_state_vals,axis=0)
                                next_state_value_array[:,:,score_gained] = new_state_vals[:,score_gained:score_gained+score_max_plus1]
                            else: 
                                new_state_vals = state_value_update[rt-1][:1]
                                next_state_value_array[:1,:,score_gained] = new_state_vals[:,score_gained:score_gained+score_max_plus1]

                    # Three throws remaining 
                    # OR two throws remaining past first round of iteration and score state is greater than 182 
                    # OR one throw remaining past first round of iteration  
                    else: ##(rt==1 and round_index>0) or (rt==2 and round_index>0 and score_state>=182) or (rt==3)
                        ## only update state of score_gained = 0
                        state_notbust_update_index = 1
                        next_state_value_array = np.zeros((token_index,61))
                        score_gained = 0
                        score_remain = score_state - score_gained
                        score_max = 60 ## always 60 here
                        score_max_plus1 = score_max + 1                    
                        ## make a copy
                        if (rt > 1):
                            if tok > 0:
                                new_state_vals = state_value_update[rt-1][tok-1:tok+1]
                                new_state_vals = np.flip(new_state_vals,axis=0)
                                next_state_value_array[:,:] = new_state_vals[:,score_gained:score_gained+score_max_plus1]
                            else: 
                                new_state_vals = state_value_update[rt-1][:1]
                                next_state_value_array[:1,:] = new_state_vals[:,score_gained:score_gained+score_max_plus1]
                        ## transit to next turn when rt=1
                        else:
                            if tok > 0: 
                                new_state_vals = optimal_value_rt3[tok-1:tok+1]
                                new_state_vals = np.flip(new_state_vals,axis=0)
                                next_state_value_array[:,:] = new_state_vals[:,score_remain:score_remain-score_max_plus1:-1]
                            else:
                                new_state_vals = optimal_value_rt3[:1]
                                next_state_value_array[:1,:] = new_state_vals[:,score_remain:score_remain-score_max_plus1:-1]
    
                    ## matrix product to compute all together
                    next_state_value_tensor = torch.from_numpy(next_state_value_array)
                    ## transit to next throw in the same turn when rt=3,2
                    if (rt > 1): 
                        # add no token expectation      
                        num_turns_tensor = prob_normalscore_tensor_nt.matmul(next_state_value_tensor[0])
                        
                        # add token expectation 
                        if tok > 0:  
                            num_turns_tensor += prob_normalscore_tensor_t.matmul(next_state_value_tensor[1])
                        
                    ## transit to next turn when rt=1
                    else:

                        # add one to indicate end of turn 
                        num_turns_tensor = 1 
                        # add no token expectation  
                        num_turns_tensor += prob_normalscore_tensor_nt.matmul(next_state_value_tensor[0])

                        if tok > 0: 
                            # add token expectation 
                            num_turns_tensor += prob_normalscore_tensor_t.matmul(next_state_value_tensor[1])
        
                    ## searching
                    temp1 = num_turns_tensor.min(axis=0)                
                    state_action_update[rt][tok][0:state_notbust_update_index] = temp1.indices.numpy()
                    state_value_update[rt][tok][0:state_notbust_update_index] =  temp1.values.numpy() 

                ## CASE 2: state which possibly bust.  score_state-score_gained<62 
                if (state_notbust_len < this_throw_state_len):
                    ## combine all bust states together 
                    state_bust_len = this_throw_state_len - state_notbust_len
                    next_state_value_array = np.zeros((token_index,61, state_bust_len))
                    for score_gained in range(state_notbust_len, this_throw_state_len):
                        ## skip infeasible state
                        if not fb.state_feasible_array[rt, score_gained]:
                            continue
                        score_remain = score_state - score_gained
                        #score_max = min(score_remain-2, 60)
                        score_max = score_remain-2 ## less than 60 here
                        score_max_plus1 = score_max + 1
                        score_gained_index = score_gained - state_notbust_len ## index off set
                        if (rt > 1):
                            if tok > 0:
                                new_state_vals = state_value_update[rt-1][tok-1:tok+1]
                                new_state_vals = np.flip(new_state_vals,axis=0)
                                next_state_value_array[:,0:score_max_plus1,score_gained_index] = new_state_vals[:,score_gained:score_gained+score_max_plus1]
                            else: 
                                new_state_vals = state_value_update[rt-1][:1]
                                next_state_value_array[:1,0:score_max_plus1,score_gained_index] = new_state_vals[:,score_gained:score_gained+score_max_plus1]
                        ## transit to next turn when rt=1
                        else:
                            if tok > 0: 
                                new_state_vals = optimal_value_rt3[tok-1:tok+1]
                                new_state_vals = np.flip(new_state_vals,axis=0)
                                next_state_value_array[:,0:score_max_plus1,score_gained_index] = new_state_vals[:,score_remain:score_remain-score_max_plus1:-1]
                            else:
                                new_state_vals = optimal_value_rt3[:1]
                                next_state_value_array[:1,0:score_max_plus1,score_gained_index] = new_state_vals[:,score_remain:score_remain-score_max_plus1:-1]

                    next_state_value_tensor = torch.from_numpy(next_state_value_array)
                    ## transit to next throw in the same turn when rt=3,2
                    if (rt > 1):  
                        # add no token expectation      
                        num_turns_tensor = prob_normalscore_tensor_nt.matmul(next_state_value_tensor[0])
                        
                        # add token expectation 
                        if tok > 0:  
                            num_turns_tensor += prob_normalscore_tensor_t.matmul(next_state_value_tensor[1])
                                          
                    ## transit to next turn when rt=1
                    else:
                        # add one to indicate end of turn 
                        num_turns_tensor = 1 
                        # add no token expectation  
                        num_turns_tensor += prob_normalscore_tensor_nt.matmul(next_state_value_tensor[0])
                        
                        if tok > 0: 
                            # add token expectation 
                            num_turns_tensor += prob_normalscore_tensor_t.matmul(next_state_value_tensor[1])                                                             
    
                    ## consider bust/finishing for each bust state seperately 
                    num_turns_array = num_turns_tensor.numpy() 
                    ## for every score gained where you go bust                
                    for score_gained in range(state_notbust_len, this_throw_state_len):
                        ## skip infeasible state
                        if not fb.state_feasible_array[rt, score_gained]:
                            continue
                        score_remain = score_state - score_gained
                        #score_max = min(score_remain-2, 60)
                        score_max = score_remain-2 ## less than 60 here
                        score_max_plus1 = score_max + 1
                        score_gained_index = score_gained - state_notbust_len
    
                        ## transit to the end of game
                        if (rt > 1):
                            if (score_remain == fb.score_DB):                        
                                num_turns_array[:729,score_gained_index] += prob_DB_nt[:729]
                                num_turns_array[729:,score_gained_index] += prob_DB_t[729:]
                            elif (score_remain <= 40 and score_remain%2==0):
                                num_turns_array[:729,score_gained_index] += prob_doublescore_dic_nt[score_remain][:729]
                                num_turns_array[729:,score_gained_index] += prob_doublescore_dic_t[score_remain][729:]
                            else:
                                pass
    
                        ## transit to bust
                        if (rt==3):
                            # In this case you really are staying in the same state becuase it's the first throw of the turn
                            # Unless you go bust with the current token policy (should never happen, but need to include to avoid choosing those actions)
                            if tok > 0: 
                                
                                # For no token, add the probbability of staying the same with nt 
                                num_turns_array[:729,score_gained_index] += prob_bust_dic_nt[score_max][:729]
                                ## solve an equation other than using the policy evaluation value (s,i=3,u=0)
                                num_turns_array[:729,score_gained_index] = num_turns_array[:729,score_gained_index] / prob_notbust_dic_nt[score_max][:729]
                                
                                # For token, add the probability of going to the new rt3 state (1 should already be included)
                                # TODO check that this makes sense - not sure about the +1 here 
                                num_turns_array[729:,score_gained_index] += prob_bust_dic_t[score_max][729:]*(optimal_value_rt3[tok-1,score_state])  ## 1 turn is already counted before
                                ## solve an equation other than using the policy evaluation value (s,i=3,u=0)
                                num_turns_array[729:,score_gained_index] = num_turns_array[729:,score_gained_index] / prob_notbust_dic_t[score_max][729:]
                                
                                
                            else: 
                                # If no token, use this logic with nt 
                                num_turns_array[:,score_gained_index] += prob_bust_dic_nt[score_max]
                                ## solve an equation other than using the policy evaluation value (s,i=3,u=0)
                                num_turns_array[:,score_gained_index] = num_turns_array[:,score_gained_index] / prob_notbust_dic_nt[score_max] 
                                
                        elif (rt==2):
                            num_turns_array[:729,score_gained_index] += prob_bust_dic_nt[score_max][:729]*(1+new_value_rt3)
                            num_turns_array[729:,score_gained_index] += prob_bust_dic_t[score_max][729:]*(1+optimal_value_rt3[tok-1,score_state])
                        else:
                            # TODO check that the +1 makes sense here 
                            num_turns_array[:729,score_gained_index] += prob_bust_dic_nt[score_max][:729]*(new_value_rt3)  ## 1 turn is already counted before
                            num_turns_array[729:,score_gained_index] += prob_bust_dic_t[score_max][729:]*(1+optimal_value_rt3[tok-1,score_state])
                    ## searching
                    temp1 = num_turns_tensor.min(axis=0)
                    state_action_update[rt][tok][state_notbust_len:this_throw_state_len] = temp1.indices.numpy()
                    state_value_update[rt][tok][state_notbust_len:this_throw_state_len] =  temp1.values.numpy()                
    
                #### finish rt=1,2,3. check improvement
                action_diff[rt][tok][:] = np.abs(state_action_update[rt][tok] - state_action[rt][tok])                                
                value_relerror[rt] = np.abs((state_value_update[rt] - state_value[rt])/state_value_update[rt]).max()
                state_action[rt][tok][:] = state_action_update[rt][tok][:]
                state_value[rt][tok][:] = state_value_update[rt][tok][:]
    
            max_action_diff = max([action_diff[1].max(), action_diff[2].max(), action_diff[3].max()])
            max_value_relerror = value_relerror.max()            
            
            if (max_action_diff < 1):
            #if max_value_relerror < iteration_relerror_limit:
                num_iteration_record[score_state] = round_index + 1
                break


    for rt in [1,2,3]:
        state_value_update[rt][tok][fb.state_infeasible[rt]] = fb.largenumber
        state_action_update[rt][tok][fb.state_infeasible[rt]] = fb.infeasible_marker
    optimal_action_index_dic[score_state] = ft.copy_numberarray_container(state_action_update)
    optimal_value_dic[score_state] = ft.copy_numberarray_container(state_value_update, new_dtype=fb.result_float_dytpe)
    optimal_value_rt3[tok][score_state] = state_value[3][tok][0]

    if tok ==0: 
        print(optimal_value_rt3)


/Users/rachael/Desktop/darts-thesis/helpers.py:506: RuntimeWarning: divide by zero encountered in divide
  num_tothrow = num_tothrow / prob_otherstate
/Users/rachael/Desktop/darts-thesis/helpers.py:506: RuntimeWarning: overflow encountered in divide
  num_tothrow = num_tothrow / prob_otherstate
/Users/rachael/Desktop/darts-thesis/helpers.py:544: RuntimeWarning: divide by zero encountered in divide
  num_tothrow = num_tothrow / prob_otherstate
/Users/rachael/Desktop/darts-thesis/helpers.py:544: RuntimeWarning: overflow encountered in divide
  num_tothrow = num_tothrow / prob_otherstate
/var/folders/jj/jcq76kp53_zchkx8b5cvhdsh0000gn/T/ipykernel_964/4234863559.py:337: RuntimeWarning: divide by zero encountered in divide
  num_turns_array[:,score_gained_index] = num_turns_array[:,score_gained_index] / prob_notbust_dic_nt[score_max]
/var/folders/jj/jcq76kp53_zchkx8b5cvhdsh0000gn/T/ipykernel_964/4234863559.py:337: RuntimeWarning: overflow encountered in divide
  num_turns_array[:,score_gaine

KeyboardInterrupt: 

In [1158]:
[aiming_grid, prob_grid_normalscore_nt, prob_grid_singlescore_nt, prob_grid_doublescore_nt, prob_grid_triplescore_nt, prob_grid_bullscore_nt] = h.load_aiming_grid(name_pa, epsilon=epsilon, data_parameter_dir=data_parameter_dir, grid_version='custom_no_tokens')
#[aiming_grid, prob_grid_normalscore_t, prob_grid_singlescore_t, prob_grid_doublescore_t, prob_grid_triplescore_t, prob_grid_bullscore_t] = h.load_aiming_grid('t', data_parameter_dir=data_parameter_dir, grid_version='custom_tokens')

prob_grid_bullscore_nt.shape

load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_no_tokens/player10_e10_gaussin_prob_grid_custom_no_tokens.pkl


(791, 2)

In [1169]:
[aiming_grid, prob_grid_normalscore_nt, prob_grid_singlescore_nt, prob_grid_doublescore_nt, prob_grid_triplescore_nt, prob_grid_bullscore_nt] = fgag.load_aiming_grid(name_pa, data_parameter_dir=data_parameter_dir, grid_version='custom_no_tokens')
#[aiming_grid, prob_grid_normalscore_t, prob_grid_singlescore_t, prob_grid_doublescore_t, prob_grid_triplescore_t, prob_grid_bullscore_t] = fgag.load_aiming_grid(name_pa, data_parameter_dir=data_parameter_dir, grid_version='custom_tokens')
prob_grid_bullscore_nt.shape
name_pa

load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_no_tokens/player10_gaussin_prob_grid_custom_no_tokens.pkl


'player10'

In [1220]:
optimal_value_dic[3]
#optimal_action_index_dic[3]
#optimal_value_rt3[:,55:65]
optimal_value_rt3[:,12:22]
#optimal_value_rt3[:,35:45]
# optimal_value_rt3[:,60:65]
# optimal_value_rt3 # [0.     0.     1.8856 ... 5.5944 5.5758 5.6116]
#optimal_value_rt3[1,27]
#optimal_value_rt3[1,21]
import pandas as pd
df = pd.DataFrame(optimal_value_rt3)
df.to_csv("test_optimal_values.csv")



In [1205]:
[aiming_grid, prob_grid_normalscore_nt_mine, prob_grid_singlescore_nt, prob_grid_doublescore_nt, prob_grid_triplescore_nt, prob_grid_bullscore_nt] = h.load_aiming_grid(name_pa, epsilon=epsilon, data_parameter_dir=data_parameter_dir, grid_version='custom_no_tokens')

[aiming_grid, prob_grid_normalscore_nt_theirs, prob_grid_singlescore_nt, prob_grid_doublescore_nt, prob_grid_triplescore_nt, prob_grid_bullscore_nt] = fgag.load_aiming_grid(name_pa, data_parameter_dir=data_parameter_dir, grid_version='custom_no_tokens')

prob_grid_normalscore_nt_mine.sum(axis=1) - prob_grid_normalscore_nt_theirs.sum(axis=1)

load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_no_tokens/player10_e1_gaussin_prob_grid_custom_no_tokens.pkl
load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_no_tokens/player10_gaussin_prob_grid_custom_no_tokens.pkl


array([-3.5565e-06,  5.5511e-16,  6.6613e-16, ...,  0.0000e+00,  0.0000e+00,  0.0000e+00])

In [1259]:
optimal_value_rt3

array([[ 0.    ,  0.    , 63.3393, ..., 80.2283, 80.2605, 80.2926],
       [ 0.    ,  0.    ,  1.    , ..., 16.985 , 17.0197, 17.0545],
       [ 0.    ,  0.    ,  1.    , ..., 14.2623, 14.2954, 14.3285],
       ...,
       [ 0.    ,  0.    ,  1.    , ...,  6.0106,  6.0427,  6.0751],
       [ 0.    ,  0.    ,  1.    , ...,  4.4165,  4.4348,  4.4822],
       [ 0.    ,  0.    ,  1.    , ...,  3.    ,  3.    ,  3.    ]])

In [1258]:
import helpers as h
import copy
tokens = 9
max_score_state = 501
tokens_plus_one = tokens + 1

epsilon = 128


debug = []

[aiming_grid, prob_grid_normalscore_nt, prob_grid_singlescore_nt, prob_grid_doublescore_nt, prob_grid_triplescore_nt, prob_grid_bullscore_nt] = h.load_aiming_grid(name_pa, epsilon=epsilon, data_parameter_dir=data_parameter_dir, grid_version='custom_no_tokens')
[aiming_grid, prob_grid_normalscore_t, prob_grid_singlescore_t, prob_grid_doublescore_t, prob_grid_triplescore_t, prob_grid_bullscore_t] = h.load_aiming_grid('t', data_parameter_dir=data_parameter_dir, grid_version='custom_tokens')

# [aiming_grid, prob_grid_normalscore_nt, prob_grid_singlescore_nt, prob_grid_doublescore_nt, prob_grid_triplescore_nt, prob_grid_bullscore_nt] = fgag.load_aiming_grid(name_pa, data_parameter_dir=data_parameter_dir, grid_version='custom_no_tokens')
# [aiming_grid, prob_grid_normalscore_t, prob_grid_singlescore_t, prob_grid_doublescore_t, prob_grid_triplescore_t, prob_grid_bullscore_t] = fgag.load_aiming_grid(name_pa, data_parameter_dir=data_parameter_dir, grid_version='custom_tokens')

## Initialize probability objects
num_aiming_location, prob_normalscore_nt, prob_doublescore_dic_nt, prob_DB_nt, prob_bust_dic_nt, prob_notbust_dic_nt, prob_normalscore_t, prob_doublescore_dic_t, prob_DB_t, prob_bust_dic_t, prob_notbust_dic_t = init_probabilities( aiming_grid, prob_grid_normalscore_nt, prob_grid_doublescore_nt, prob_grid_bullscore_nt, prob_grid_normalscore_t, prob_grid_doublescore_t, prob_grid_bullscore_t)    

# Combined probabilities  
prob_grid_normalscore_comb = np.zeros(prob_grid_normalscore_t.shape)
prob_grid_normalscore_comb[:729,:] = prob_grid_normalscore_nt[:729,:]
prob_grid_normalscore_comb[729:] = prob_grid_normalscore_t[729:]

prob_grid_doublescore_comb = np.zeros(prob_grid_doublescore_t.shape)
prob_grid_doublescore_comb[:729,:] = prob_grid_doublescore_nt[:729,:]
prob_grid_doublescore_comb[729:] = prob_grid_doublescore_t[729:]

prob_grid_bullscore_comb = np.zeros(prob_grid_bullscore_t.shape)
prob_grid_bullscore_comb[:729,:] = prob_grid_bullscore_nt[:729,:]
prob_grid_bullscore_comb[729:] = prob_grid_bullscore_t[729:]

# Initialize joint probability notbust dictionary 
prob_notbust_dic = {}
for score_max in prob_notbust_dic_t.keys():
    prob_notbust_dic[score_max] = np.zeros(prob_notbust_dic_t[0].shape[0]) # 791 
    prob_notbust_dic[score_max][:729] += prob_notbust_dic_nt[score_max][:729] 
    prob_notbust_dic[score_max][729:] += prob_notbust_dic_t[score_max][729:]

# Intialize joint probability bust dictionary 
prob_bust_dic = {}
for score_max in prob_notbust_dic_t.keys():
    prob_bust_dic[score_max] = np.zeros(prob_bust_dic_t[0].shape[0]) # 791 
    prob_bust_dic[score_max][:729] += prob_bust_dic_nt[score_max][:729] 
    prob_bust_dic[score_max][729:] += prob_bust_dic_t[score_max][729:]


prob_normalscore_tensor_nt = torch.from_numpy(prob_normalscore_nt)
prob_normalscore_tensor_t = torch.from_numpy(prob_normalscore_t)
prob_normalscore_tensor = torch.from_numpy(prob_grid_normalscore_comb)

# prob_doublescore_dic = prob_doublescore_dic_nt
# prob_DB = prob_DB_nt
# prob_bust_dic = prob_bust_dic_nt
# prob_notbust_dic = prob_notbust_dic_nt

iteration_round_limit = 20

optimal_value_rt3 = np.zeros((tokens_plus_one,502)) #vector: optimal value for the beginning state of each turn (rt=3)
optimal_value_dic = {} ## first key: score=0,2,...,501, second key: remaining throws=3,2,1
optimal_action_index_dic = {}
num_iteration_record = np.zeros(502, dtype=np.int32)

state_len_vector = np.zeros(4, dtype=np.int32)
state_value  = [None]  ## optimal value (expected # of turns to finish the game) for each state in the current playing turn
state_action = [None]  ## aimming locations for for each state in the current playing turn
action_diff  = [None]
value_relerror = np.zeros(4)

for rt in [1,2,3]:
    ## for rt=3: possible score_gained = 0
    ## for rt=2: possible score_gained = 0,1,...,60
    ## for rt=1: possible score_gained = 0,1,...,120
    this_throw_state_len = fb.maxhitscore*(3-rt) + 1
    state_value.append(np.ones((tokens_plus_one,this_throw_state_len))*fb.largenumber)
    state_action.append(np.ones((tokens_plus_one,this_throw_state_len), np.int32)*fb.infeasible_marker)
    action_diff.append(np.ones((tokens_plus_one,this_throw_state_len)))
state_value_update = ft.copy_numberarray_container(state_value)
state_action_update = ft.copy_numberarray_container(state_action)

## use no_turn policy as the initial policy
#[noturn_optimal_value, noturn_optimal_action_index] = solve_dp_noturn(aiming_grid, prob_grid_normalscore, prob_grid_doublescore, prob_grid_bullscore)
[noturn_optimal_value, noturn_optimal_action_index] = h.solve_dp_noturn_tokens(aiming_grid, prob_grid_normalscore_nt, prob_grid_normalscore_t, tokens = tokens, prob_grid_doublescore=prob_grid_doublescore_nt, prob_grid_bullscore=prob_grid_bullscore_nt, prob_grid_doublescore_dic=None, prob_grid_doublescore_t=prob_grid_doublescore_t, prob_grid_bullscore_t=prob_grid_bullscore_t, prob_grid_doublescore_dic_t=None)

t1 = time.time()
for score_state in range(2,max_score_state+1):

    for tok in range(tokens_plus_one):
        debug.append((score_state,tok,copy.deepcopy(state_value_update)))
        ## initialization 
        for rt in [1,2,3]:
        
            ## for rt=3: score_gained = 0
            ## for rt=2: score_gained = 0,1,...,min(s-2,60)
            ## for rt=1: score_gained = 0,1,...,min(s-2,120)
            this_throw_state_len = min(score_state-2, fb.maxhitscore*(3-rt)) + 1
            state_len_vector[rt] = this_throw_state_len
                    
            ## initialize the starting policy: 
            ## use no_turn action in (s, i, u=0)
            ## use turn action (s-1, i, u-1) in (s, i, u!=0) if (s-1, i, u-1) is feasible state
            state_action[rt][tok][0] = noturn_optimal_action_index[tok][score_state]            
            for score_gained in range(1,this_throw_state_len):                
                if fb.state_feasible_array[rt, score_gained]:  ## if True
                    if fb.state_feasible_array[rt, score_gained-1]:
                        state_action[rt][tok][score_gained] = optimal_action_index_dic[score_state-1][rt][tok][score_gained-1]
                    else:                        
                        state_action[rt][tok][score_gained] = noturn_optimal_action_index[tok][score_state-score_gained]
                else:
                    state_action[rt][tok][score_gained] = fb.infeasible_marker
    
    # for tok in range(tokens_plus_one):

        # policy iteration
        for round_index in range(iteration_round_limit):

            ## --------------------------------------------- ##
            ##                 POLICY EVALUATION
            ## --------------------------------------------- ##
            rt = 3
            score_gained = 0
            score_max_turn = min(score_state-2, 3*fb.maxhitscore)

            # Get the transit probabilities 
            prob_turn_transit = solve_turn_transit_probability_fast_token(score_state, state_action, tok, prob_grid_normalscore_nt, prob_grid_doublescore_nt, prob_grid_bullscore_nt, prob_bust_dic_nt,prob_grid_normalscore_t, prob_grid_doublescore_t, prob_grid_bullscore_t, prob_bust_dic_t)
            
            tokens_max_turn = min(prob_turn_transit['bust'].shape[0]-1,tok)

            # Make updates below 
            prob_turn_zeroscore = prob_turn_transit['bust'][0] + prob_turn_transit['score'][0,0]

            new_value_rt3 = 1 

            # Don't use a token 
            new_value_rt3 += np.dot(prob_turn_transit['score'][0,1:], optimal_value_rt3[tok,score_state-1:score_state-score_max_turn-1:-1])
            
            # Use at least one token 
            if tok > 0: 

                # Go to subsequent score after using at least one token
                for i in range(1,tokens_max_turn+1):
                    #new_value_rt3 += np.dot(prob_turn_transit['score'][0:tokens_max_turn+1,:][i], optimal_value_rt3[tok-tokens_max_turn:tok+1,score_state:score_state-score_max_turn-1:-1][i])

                    new_value_rt3 += np.dot(np.flip(prob_turn_transit['score'][0:tokens_max_turn+1,:],axis=0)[i], optimal_value_rt3[tok-tokens_max_turn:tok+1,score_state:score_state-score_max_turn-1:-1][i])
               
                # Go bust after using at least one token 
                new_value_rt3 += np.dot(np.flip(prob_turn_transit['bust'][1:tokens_max_turn+1]) , optimal_value_rt3[tok-tokens_max_turn+1:tok+1,score_state])

            # Normalize over probability of moving to a new state 
            new_value_rt3 = new_value_rt3 / (1-prob_turn_zeroscore)
            new_value_rt3

            state_value_update[rt][tok][score_gained] = new_value_rt3
            optimal_value_rt3[tok][score_state] = new_value_rt3
            #print('evaluate rt3 value= {}'.format(new_value_rt3)

            ## --------------------------------------------- ##
            ##                 POLICY IMPROVEMENT
            ## --------------------------------------------- ## 

            for rt in [1,2,3]: 

                this_throw_state_len = state_len_vector[rt]
                state_notbust_len =  max(min(score_state-61, this_throw_state_len),0)
                token_index = min(2,tok+1)

                ## CASE 1: state which can not bust.  score_state-score_gained>=62 
                if (state_notbust_len > 0):
                    # One throw remaining and first round of iteration 
                    if (rt==1 and round_index==0):
                        ## combine all non-bust states together 
                        state_notbust_update_index = state_notbust_len                    
                        next_state_value_array_nt = np.zeros((61, state_notbust_len))   
                        next_state_value_array_t = np.zeros((61, state_notbust_len))             
                        for score_gained in range(state_notbust_len):
                            ## skip infeasible state
                            if not fb.state_feasible_array[rt, score_gained]:
                                continue
                            score_remain = score_state - score_gained
                            score_max = 60 ## always 60 here
                            score_max_plus1 = score_max + 1

                            new_state_vals_nt = optimal_value_rt3[tok]
                            next_state_value_array_nt[:,score_gained] = new_state_vals_nt[score_remain:score_remain-score_max_plus1:-1]

                            if tok > 0: 
                                new_state_vals_t = optimal_value_rt3[tok-1]
                                next_state_value_array_t[:,score_gained] = new_state_vals_t[score_remain:score_remain-score_max_plus1:-1]
                                
                    # Two throws remaining and first round of iteration 
                    elif (rt==2 and (round_index==0 or score_state<182)):
                        ## combine all non-bust states together 
                        state_notbust_update_index = state_notbust_len
                        next_state_value_array_nt = np.zeros((61, state_notbust_len))      
                        next_state_value_array_t = np.zeros((61, state_notbust_len))                      
                        for score_gained in range(state_notbust_len):
                            ## skip infeasible state
                            if not fb.state_feasible_array[rt, score_gained]:
                                continue
                            score_remain = score_state - score_gained
                            score_max = 60 ## always 60 here
                            score_max_plus1 = score_max + 1

                            new_state_vals_nt = state_value_update[rt-1][tok]
                            next_state_value_array_nt[:,score_gained] = new_state_vals_nt[score_gained:score_gained+score_max_plus1]

                            if tok > 0: 
                                new_state_vals_t = state_value_update[rt-1][tok-1]
                                next_state_value_array_t[:,score_gained] = new_state_vals_t[score_gained:score_gained+score_max_plus1]

                                
                    # Three throws remaining 
                    # OR two throws remaining past first round of iteration and score state is greater than 182 
                    # OR one throw remaining past first round of iteration  
                    else: ##(rt==1 and round_index>0) or (rt==2 and round_index>0 and score_state>=182) or (rt==3)
                        ## only update state of score_gained = 0
                        state_notbust_update_index = 1
                        next_state_value_array_nt= np.zeros((61))
                        next_state_value_array_t= np.zeros((61))
                        score_gained = 0
                        score_remain = score_state - score_gained
                        score_max = 60 ## always 60 here
                        score_max_plus1 = score_max + 1                    
                        ## make a copy
                        if (rt > 1):

                            new_state_vals_nt = state_value_update[rt-1][tok]
                            next_state_value_array_nt[:] = new_state_vals_nt[score_gained:score_gained+score_max_plus1]
                            
                            if tok > 0:
                                new_state_vals_t = state_value_update[rt-1][tok-1]
                                next_state_value_array_t[:] = new_state_vals_t[score_gained:score_gained+score_max_plus1]

                        ## transit to next turn when rt=1
                        else:
                            new_state_vals_nt = optimal_value_rt3[tok]
                            next_state_value_array_nt[:] = new_state_vals_nt[score_remain:score_remain-score_max_plus1:-1]
                            
                            if tok > 0: 
                                new_state_vals_t = optimal_value_rt3[tok-1]
                                next_state_value_array_t[:] = new_state_vals_t[score_remain:score_remain-score_max_plus1:-1]
                                
    
                    ## matrix product to compute all together
                    next_state_value_tensor_nt = torch.from_numpy(next_state_value_array_nt)
                    next_state_value_tensor_t = torch.from_numpy(next_state_value_array_t)
                    ## transit to next throw in the same turn when rt=3,2
                    if (rt > 1): 

                        # add one to indicate end of turn 
                        try: 
                            num_turns_tensor = torch.zeros((791,next_state_value_array_nt.shape[1]))
                        except:
                            num_turns_tensor = torch.zeros(791)
                        
                        # if we have tokens 
                        if tok > 0:  

                            # use no token probabilities for no token actions 
                            num_turns_tensor[:729] += prob_normalscore_tensor_nt[:729].matmul(next_state_value_tensor_nt)
                            # use token probabilities for token actions 
                            num_turns_tensor[729:791] += prob_normalscore_tensor_t[729:].matmul(next_state_value_tensor_t)

                        # if we don't have tokens   
                        else:
                            # add no token expectation --> index is 1 because we keep the same # of tokens       
                            num_turns_tensor = prob_normalscore_tensor_nt.matmul(next_state_value_tensor_nt)
                        
                    ## transit to next turn when rt=1
                    else:

                        # add one to indicate end of turn 
                        try: 
                            num_turns_tensor = torch.ones((791,next_state_value_array_nt.shape[1]))
                        except:
                            num_turns_tensor = torch.ones(791)
                    
                        # if we have tokens 
                        if tok > 0: 
                            # use no token probabilities for no token actions
                            num_turns_tensor[:729] += prob_normalscore_tensor_nt[:729].matmul(next_state_value_tensor_nt)
                            
                            # use token probabilities for token actions 
                            num_turns_tensor[729:791] += prob_normalscore_tensor_t[729:].matmul(next_state_value_tensor_t)
                        else: 
                            # add no token expectation  
                            num_turns_tensor += prob_normalscore_tensor_nt.matmul(next_state_value_tensor_nt)

                    ## searching
                    #temp1 = num_turns_tensor.min(axis=0) 
                    if tok == 0: 
                        temp1 = num_turns_tensor[:729].min(axis=0)
                    else: 
                        temp1 = num_turns_tensor.min(axis=0)               
                    state_action_update[rt][tok][0:state_notbust_update_index] = temp1.indices.numpy()
                    state_value_update[rt][tok][0:state_notbust_update_index] =  temp1.values.numpy() 

                ## CASE 2: state which possibly bust.  score_state-score_gained<62 
                if (state_notbust_len < this_throw_state_len):
                    ## combine all bust states together 
                    state_bust_len = this_throw_state_len - state_notbust_len
                    next_state_value_array_nt = np.zeros((61, state_bust_len))
                    next_state_value_array_t = np.zeros((61, state_bust_len))
                    for score_gained in range(state_notbust_len, this_throw_state_len):
                        ## skip infeasible state
                        if not fb.state_feasible_array[rt, score_gained]:
                            continue
                        score_remain = score_state - score_gained
                        #score_max = min(score_remain-2, 60)
                        score_max = score_remain-2 ## less than 60 here
                        score_max_plus1 = score_max + 1
                        score_gained_index = score_gained - state_notbust_len ## index off set
                        if (rt > 1):

                            new_state_vals_nt = state_value_update[rt-1][tok]
                            next_state_value_array_nt[0:score_max_plus1,score_gained_index] = new_state_vals_nt[score_gained:score_gained+score_max_plus1]

                            if tok > 0: 
                                new_state_vals_t = state_value_update[rt-1][tok-1]
                                next_state_value_array_t[0:score_max_plus1,score_gained_index] = new_state_vals_t[score_gained:score_gained+score_max_plus1]

                        ## transit to next turn when rt=1
                        else:

                            new_state_vals_nt = optimal_value_rt3[tok]
                            next_state_value_array_nt[0:score_max_plus1,score_gained_index] = new_state_vals_nt[score_remain:score_remain-score_max_plus1:-1]

                            
                            if tok > 0: 
                                new_state_vals_t = optimal_value_rt3[tok-1]
                                next_state_value_array_t[0:score_max_plus1,score_gained_index] = new_state_vals_t[score_remain:score_remain-score_max_plus1:-1]
                                
                    next_state_value_tensor_nt = torch.from_numpy(next_state_value_array_nt)
                    next_state_value_tensor_t = torch.from_numpy(next_state_value_array_t)
                    
                    ## transit to next throw in the same turn when rt=3,2
                    if (rt > 1):  

                        # initialized
                        try: 
                            num_turns_tensor = torch.zeros((791,next_state_value_array_nt.shape[1]))
                        except:
                            num_turns_tensor = torch.zeros(791)
                    
                        # if we have tokens 
                        if tok > 0: 
                            # use no token probabilities for no token actions
                            num_turns_tensor[:729] += prob_normalscore_tensor_nt[:729].matmul(next_state_value_tensor_nt)
                            
                            # use token probabilities for token actions 
                            num_turns_tensor[729:791] += prob_normalscore_tensor_t[729:].matmul(next_state_value_tensor_t)
                        else: 
                            # add no token expectation  
                            num_turns_tensor += prob_normalscore_tensor_nt.matmul(next_state_value_tensor_nt)

                                          
                    ## transit to next turn when rt=1
                    else:

                        # add one to indicate end of turn 
                        try: 
                            num_turns_tensor = torch.ones((791,next_state_value_array_nt.shape[1]))
                        except:
                            num_turns_tensor = torch.ones(791)
                    
                        # if we have tokens 
                        if tok > 0: 
                            # use no token probabilities for no token actions
                            num_turns_tensor[:729] += prob_normalscore_tensor_nt[:729].matmul(next_state_value_tensor_nt)
                            
                            # use token probabilities for token actions 
                            num_turns_tensor[729:791] += prob_normalscore_tensor_t[729:].matmul(next_state_value_tensor_t)
                        else: 
                            # add no token expectation  
                            num_turns_tensor += prob_normalscore_tensor_nt.matmul(next_state_value_tensor_nt)

                        # # add one to indicate end of turn 
                        # num_turns_tensor = 1 
                        # # add no token expectation  
                        # num_turns_tensor += prob_normalscore_tensor_nt.matmul(next_state_value_tensor[0])
                        
                        # if tok > 0: 
                        #     # add token expectation 
                        #     num_turns_tensor += prob_normalscore_tensor_t.matmul(next_state_value_tensor[1])                                                             
    
                    ## consider bust/finishing for each bust state seperately 
                    num_turns_array = num_turns_tensor.numpy() 
                    ## for every score gained where you go bust                
                    for score_gained in range(state_notbust_len, this_throw_state_len):
                        ## skip infeasible state
                        if not fb.state_feasible_array[rt, score_gained]:
                            continue
                        score_remain = score_state - score_gained
                        #score_max = min(score_remain-2, 60)
                        score_max = score_remain-2 ## less than 60 here
                        score_max_plus1 = score_max + 1
                        score_gained_index = score_gained - state_notbust_len
                        
                        ## transit to the end of game
                        if (rt > 1):
                            if (score_remain == fb.score_DB):                        
                                num_turns_array[:729,score_gained_index] += prob_DB_nt[:729]
                                num_turns_array[729:,score_gained_index] += prob_DB_t[729:]
                            elif (score_remain <= 40 and score_remain%2==0):
                                num_turns_array[:729,score_gained_index] += prob_doublescore_dic_nt[score_remain][:729]
                                num_turns_array[729:,score_gained_index] += prob_doublescore_dic_t[score_remain][729:]
                            else:
                                pass
    
                        ## transit to bust
                        if (rt==3):
                            # In this case you really are staying in the same state becuase it's the first throw of the turn
                            # Unless you go bust with the current token policy (should never happen, but need to include to avoid choosing those actions)
                            if tok > 0: 
                                
                                # For no token, add the probbability of staying the same with nt 
                                num_turns_array[:729,score_gained_index] += prob_bust_dic_nt[score_max][:729]
                                ## solve an equation other than using the policy evaluation value (s,i=3,u=0)
                                num_turns_array[:729,score_gained_index] = num_turns_array[:729,score_gained_index] / prob_notbust_dic_nt[score_max][:729]
                                
                                # For token, add the probability of going to the new rt3 state (1 should already be included)
                                # TODO check that this makes sense - not sure about the +1 here 
                                num_turns_array[729:,score_gained_index] += prob_bust_dic_t[score_max][729:]*(1+optimal_value_rt3[tok-1,score_state])  ## 1 turn is already counted before
                                ## solve an equation other than using the policy evaluation value (s,i=3,u=0)
                                num_turns_array[729:,score_gained_index] = num_turns_array[729:,score_gained_index] / prob_notbust_dic_t[score_max][729:]
                                
                                
                            else: 
                                # If no token, use this logic with nt 
                                num_turns_array[:,score_gained_index] += prob_bust_dic_nt[score_max]
                                ## solve an equation other than using the policy evaluation value (s,i=3,u=0)
                                num_turns_array[:,score_gained_index] = num_turns_array[:,score_gained_index] / prob_notbust_dic_nt[score_max] 
                                
                        elif (rt==2):
                            num_turns_array[:729,score_gained_index] += prob_bust_dic_nt[score_max][:729]*(1+new_value_rt3)
                            num_turns_array[729:,score_gained_index] += prob_bust_dic_t[score_max][729:]*(1+optimal_value_rt3[tok-1,score_state])
                        else:
                            # TODO check that the +1 makes sense here 
                            num_turns_array[:729,score_gained_index] += prob_bust_dic_nt[score_max][:729]*(new_value_rt3)  ## 1 turn is already counted before
                            num_turns_array[729:,score_gained_index] += prob_bust_dic_t[score_max][729:]*(1+optimal_value_rt3[tok-1,score_state])
                    
                    ## searching
                    if tok == 0: 
                        temp1 = num_turns_tensor[:729].min(axis=0)
                    else: 
                        temp1 = num_turns_tensor.min(axis=0)
                    state_action_update[rt][tok][state_notbust_len:this_throw_state_len] = temp1.indices.numpy()
                    state_value_update[rt][tok][state_notbust_len:this_throw_state_len] =  temp1.values.numpy()                
    
                #### finish rt=1,2,3. check improvement
                action_diff[rt][tok][:] = np.abs(state_action_update[rt][tok] - state_action[rt][tok])                                
                value_relerror[rt] = np.abs((state_value_update[rt] - state_value[rt])/state_value_update[rt]).max()
                state_action[rt][tok][:] = state_action_update[rt][tok][:]
                state_value[rt][tok][:] = state_value_update[rt][tok][:]
    
            max_action_diff = max([action_diff[1].max(), action_diff[2].max(), action_diff[3].max()])
            max_value_relerror = value_relerror.max()            
            
            if (max_action_diff < 1):
            #if max_value_relerror < iteration_relerror_limit:
                num_iteration_record[score_state] = round_index + 1
                break


        for rt in [1,2,3]:
            state_value_update[rt][tok][fb.state_infeasible[rt]] = fb.largenumber
            state_action_update[rt][tok][fb.state_infeasible[rt]] = fb.infeasible_marker
        optimal_action_index_dic[score_state] = ft.copy_numberarray_container(state_action_update)
        optimal_value_dic[score_state] = ft.copy_numberarray_container(state_value_update, new_dtype=fb.result_float_dytpe)
        optimal_value_rt3[tok][score_state] = state_value[3][tok][0]

        # if tok ==0: 
        #     print(optimal_value_rt3)


load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_no_tokens/player10_e128_gaussin_prob_grid_custom_no_tokens.pkl
load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_tokens/t_gaussin_prob_grid_custom_tokens.pkl


/Users/rachael/Desktop/darts-thesis/helpers.py:506: RuntimeWarning: divide by zero encountered in divide
  num_tothrow = num_tothrow / prob_otherstate
/Users/rachael/Desktop/darts-thesis/helpers.py:544: RuntimeWarning: divide by zero encountered in divide
  num_tothrow = num_tothrow / prob_otherstate
/var/folders/jj/jcq76kp53_zchkx8b5cvhdsh0000gn/T/ipykernel_964/3997785353.py:426: RuntimeWarning: divide by zero encountered in divide
  num_turns_array[729:,score_gained_index] = num_turns_array[729:,score_gained_index] / prob_notbust_dic_t[score_max][729:]
/var/folders/jj/jcq76kp53_zchkx8b5cvhdsh0000gn/T/ipykernel_964/3997785353.py:453: RuntimeWarning: divide by zero encountered in divide
  value_relerror[rt] = np.abs((state_value_update[rt] - state_value[rt])/state_value_update[rt]).max()
/var/folders/jj/jcq76kp53_zchkx8b5cvhdsh0000gn/T/ipykernel_964/3997785353.py:453: RuntimeWarning: invalid value encountered in divide
  value_relerror[rt] = np.abs((state_value_update[rt] - state_value

In [ ]:
import helpers as h
import copy
tokens = 9



epsilon = 128



[aiming_grid, prob_grid_normalscore_nt, prob_grid_singlescore_nt, prob_grid_doublescore_nt, prob_grid_triplescore_nt, prob_grid_bullscore_nt] = h.load_aiming_grid(name_pa, epsilon=epsilon, data_parameter_dir=data_parameter_dir, grid_version='custom_no_tokens')
[aiming_grid, prob_grid_normalscore_t, prob_grid_singlescore_t, prob_grid_doublescore_t, prob_grid_triplescore_t, prob_grid_bullscore_t] = h.load_aiming_grid('t', data_parameter_dir=data_parameter_dir, grid_version='custom_tokens')

def solve_dp_turn_token(tokens, aiming_grid, prob_grid_normalscore_nt, prob_grid_singlescore_nt, prob_grid_doublescore_nt, prob_grid_triplescore_nt, prob_grid_bullscore_nt,prob_grid_normalscore_t, prob_grid_singlescore_t, prob_grid_doublescore_t, prob_grid_triplescore_t, prob_grid_bullscore_t):

    tokens_plus_one = tokens + 1

    ## Initialize probability objects
    num_aiming_location, prob_normalscore_nt, prob_doublescore_dic_nt, prob_DB_nt, prob_bust_dic_nt, prob_notbust_dic_nt, prob_normalscore_t, prob_doublescore_dic_t, prob_DB_t, prob_bust_dic_t, prob_notbust_dic_t = init_probabilities( aiming_grid, prob_grid_normalscore_nt, prob_grid_doublescore_nt, prob_grid_bullscore_nt, prob_grid_normalscore_t, prob_grid_doublescore_t, prob_grid_bullscore_t)    

    # Combined probabilities  
    prob_grid_normalscore_comb = np.zeros(prob_grid_normalscore_t.shape)
    prob_grid_normalscore_comb[:729,:] = prob_grid_normalscore_nt[:729,:]
    prob_grid_normalscore_comb[729:] = prob_grid_normalscore_t[729:]

    prob_grid_doublescore_comb = np.zeros(prob_grid_doublescore_t.shape)
    prob_grid_doublescore_comb[:729,:] = prob_grid_doublescore_nt[:729,:]
    prob_grid_doublescore_comb[729:] = prob_grid_doublescore_t[729:]

    prob_grid_bullscore_comb = np.zeros(prob_grid_bullscore_t.shape)
    prob_grid_bullscore_comb[:729,:] = prob_grid_bullscore_nt[:729,:]
    prob_grid_bullscore_comb[729:] = prob_grid_bullscore_t[729:]

    # Initialize joint probability notbust dictionary 
    prob_notbust_dic = {}
    for score_max in prob_notbust_dic_t.keys():
        prob_notbust_dic[score_max] = np.zeros(prob_notbust_dic_t[0].shape[0]) # 791 
        prob_notbust_dic[score_max][:729] += prob_notbust_dic_nt[score_max][:729] 
        prob_notbust_dic[score_max][729:] += prob_notbust_dic_t[score_max][729:]

    # Intialize joint probability bust dictionary 
    prob_bust_dic = {}
    for score_max in prob_notbust_dic_t.keys():
        prob_bust_dic[score_max] = np.zeros(prob_bust_dic_t[0].shape[0]) # 791 
        prob_bust_dic[score_max][:729] += prob_bust_dic_nt[score_max][:729] 
        prob_bust_dic[score_max][729:] += prob_bust_dic_t[score_max][729:]


    prob_normalscore_tensor_nt = torch.from_numpy(prob_normalscore_nt)
    prob_normalscore_tensor_t = torch.from_numpy(prob_normalscore_t)
    prob_normalscore_tensor = torch.from_numpy(prob_grid_normalscore_comb)

    # prob_doublescore_dic = prob_doublescore_dic_nt
    # prob_DB = prob_DB_nt
    # prob_bust_dic = prob_bust_dic_nt
    # prob_notbust_dic = prob_notbust_dic_nt

    iteration_round_limit = 20

    optimal_value_rt3 = np.zeros((tokens_plus_one,502)) #vector: optimal value for the beginning state of each turn (rt=3)
    optimal_value_dic = {} ## first key: score=0,2,...,501, second key: remaining throws=3,2,1
    optimal_action_index_dic = {}
    num_iteration_record = np.zeros(502, dtype=np.int32)

    state_len_vector = np.zeros(4, dtype=np.int32)
    state_value  = [None]  ## optimal value (expected # of turns to finish the game) for each state in the current playing turn
    state_action = [None]  ## aimming locations for for each state in the current playing turn
    action_diff  = [None]
    value_relerror = np.zeros(4)

    for rt in [1,2,3]:
        ## for rt=3: possible score_gained = 0
        ## for rt=2: possible score_gained = 0,1,...,60
        ## for rt=1: possible score_gained = 0,1,...,120
        this_throw_state_len = fb.maxhitscore*(3-rt) + 1
        state_value.append(np.ones((tokens_plus_one,this_throw_state_len))*fb.largenumber)
        state_action.append(np.ones((tokens_plus_one,this_throw_state_len), np.int32)*fb.infeasible_marker)
        action_diff.append(np.ones((tokens_plus_one,this_throw_state_len)))
    state_value_update = ft.copy_numberarray_container(state_value)
    state_action_update = ft.copy_numberarray_container(state_action)

    ## use no_turn policy as the initial policy
    #[noturn_optimal_value, noturn_optimal_action_index] = solve_dp_noturn(aiming_grid, prob_grid_normalscore, prob_grid_doublescore, prob_grid_bullscore)
    [noturn_optimal_value, noturn_optimal_action_index] = h.solve_dp_noturn_tokens(aiming_grid, prob_grid_normalscore_nt, prob_grid_normalscore_t, tokens = tokens, prob_grid_doublescore=prob_grid_doublescore_nt, prob_grid_bullscore=prob_grid_bullscore_nt, prob_grid_doublescore_dic=None, prob_grid_doublescore_t=prob_grid_doublescore_t, prob_grid_bullscore_t=prob_grid_bullscore_t, prob_grid_doublescore_dic_t=None)

    t1 = time.time()
    for score_state in range(2,502):

        for tok in range(tokens_plus_one):
            
            ## initialization 
            for rt in [1,2,3]:
            
                ## for rt=3: score_gained = 0
                ## for rt=2: score_gained = 0,1,...,min(s-2,60)
                ## for rt=1: score_gained = 0,1,...,min(s-2,120)
                this_throw_state_len = min(score_state-2, fb.maxhitscore*(3-rt)) + 1
                state_len_vector[rt] = this_throw_state_len
                        
                ## initialize the starting policy: 
                ## use no_turn action in (s, i, u=0)
                ## use turn action (s-1, i, u-1) in (s, i, u!=0) if (s-1, i, u-1) is feasible state
                state_action[rt][tok][0] = noturn_optimal_action_index[tok][score_state]            
                for score_gained in range(1,this_throw_state_len):                
                    if fb.state_feasible_array[rt, score_gained]:  ## if True
                        if fb.state_feasible_array[rt, score_gained-1]:
                            state_action[rt][tok][score_gained] = optimal_action_index_dic[score_state-1][rt][tok][score_gained-1]
                        else:                        
                            state_action[rt][tok][score_gained] = noturn_optimal_action_index[tok][score_state-score_gained]
                    else:
                        state_action[rt][tok][score_gained] = fb.infeasible_marker
        
        # for tok in range(tokens_plus_one):

            # policy iteration
            for round_index in range(iteration_round_limit):

                ## --------------------------------------------- ##
                ##                 POLICY EVALUATION
                ## --------------------------------------------- ##
                rt = 3
                score_gained = 0
                score_max_turn = min(score_state-2, 3*fb.maxhitscore)

                # Get the transit probabilities 
                prob_turn_transit = solve_turn_transit_probability_fast_token(score_state, state_action, tok, prob_grid_normalscore_nt, prob_grid_doublescore_nt, prob_grid_bullscore_nt, prob_bust_dic_nt,prob_grid_normalscore_t, prob_grid_doublescore_t, prob_grid_bullscore_t, prob_bust_dic_t)
                
                tokens_max_turn = min(prob_turn_transit['bust'].shape[0]-1,tok)

                # Make updates below 
                prob_turn_zeroscore = prob_turn_transit['bust'][0] + prob_turn_transit['score'][0,0]

                new_value_rt3 = 1 

                # Don't use a token 
                new_value_rt3 += np.dot(prob_turn_transit['score'][0,1:], optimal_value_rt3[tok,score_state-1:score_state-score_max_turn-1:-1])
                
                # Use at least one token 
                if tok > 0: 

                    # Go to subsequent score after using at least one token
                    for i in range(1,tokens_max_turn+1):
                        #new_value_rt3 += np.dot(prob_turn_transit['score'][0:tokens_max_turn+1,:][i], optimal_value_rt3[tok-tokens_max_turn:tok+1,score_state:score_state-score_max_turn-1:-1][i])

                        new_value_rt3 += np.dot(np.flip(prob_turn_transit['score'][0:tokens_max_turn+1,:],axis=0)[i], optimal_value_rt3[tok-tokens_max_turn:tok+1,score_state:score_state-score_max_turn-1:-1][i])
                
                    # Go bust after using at least one token 
                    new_value_rt3 += np.dot(np.flip(prob_turn_transit['bust'][1:tokens_max_turn+1]) , optimal_value_rt3[tok-tokens_max_turn+1:tok+1,score_state])

                # Normalize over probability of moving to a new state 
                new_value_rt3 = new_value_rt3 / (1-prob_turn_zeroscore)
                new_value_rt3

                state_value_update[rt][tok][score_gained] = new_value_rt3
                optimal_value_rt3[tok][score_state] = new_value_rt3
                #print('evaluate rt3 value= {}'.format(new_value_rt3)

                ## --------------------------------------------- ##
                ##                 POLICY IMPROVEMENT
                ## --------------------------------------------- ## 

                for rt in [1,2,3]: 

                    this_throw_state_len = state_len_vector[rt]
                    state_notbust_len =  max(min(score_state-61, this_throw_state_len),0)
                    token_index = min(2,tok+1)

                    ## CASE 1: state which can not bust.  score_state-score_gained>=62 
                    if (state_notbust_len > 0):
                        # One throw remaining and first round of iteration 
                        if (rt==1 and round_index==0):
                            ## combine all non-bust states together 
                            state_notbust_update_index = state_notbust_len                    
                            next_state_value_array_nt = np.zeros((61, state_notbust_len))   
                            next_state_value_array_t = np.zeros((61, state_notbust_len))             
                            for score_gained in range(state_notbust_len):
                                ## skip infeasible state
                                if not fb.state_feasible_array[rt, score_gained]:
                                    continue
                                score_remain = score_state - score_gained
                                score_max = 60 ## always 60 here
                                score_max_plus1 = score_max + 1

                                new_state_vals_nt = optimal_value_rt3[tok]
                                next_state_value_array_nt[:,score_gained] = new_state_vals_nt[score_remain:score_remain-score_max_plus1:-1]

                                if tok > 0: 
                                    new_state_vals_t = optimal_value_rt3[tok-1]
                                    next_state_value_array_t[:,score_gained] = new_state_vals_t[score_remain:score_remain-score_max_plus1:-1]
                                    
                        # Two throws remaining and first round of iteration 
                        elif (rt==2 and (round_index==0 or score_state<182)):
                            ## combine all non-bust states together 
                            state_notbust_update_index = state_notbust_len
                            next_state_value_array_nt = np.zeros((61, state_notbust_len))      
                            next_state_value_array_t = np.zeros((61, state_notbust_len))                      
                            for score_gained in range(state_notbust_len):
                                ## skip infeasible state
                                if not fb.state_feasible_array[rt, score_gained]:
                                    continue
                                score_remain = score_state - score_gained
                                score_max = 60 ## always 60 here
                                score_max_plus1 = score_max + 1

                                new_state_vals_nt = state_value_update[rt-1][tok]
                                next_state_value_array_nt[:,score_gained] = new_state_vals_nt[score_gained:score_gained+score_max_plus1]

                                if tok > 0: 
                                    new_state_vals_t = state_value_update[rt-1][tok-1]
                                    next_state_value_array_t[:,score_gained] = new_state_vals_t[score_gained:score_gained+score_max_plus1]

                                    
                        # Three throws remaining 
                        # OR two throws remaining past first round of iteration and score state is greater than 182 
                        # OR one throw remaining past first round of iteration  
                        else: ##(rt==1 and round_index>0) or (rt==2 and round_index>0 and score_state>=182) or (rt==3)
                            ## only update state of score_gained = 0
                            state_notbust_update_index = 1
                            next_state_value_array_nt= np.zeros((61))
                            next_state_value_array_t= np.zeros((61))
                            score_gained = 0
                            score_remain = score_state - score_gained
                            score_max = 60 ## always 60 here
                            score_max_plus1 = score_max + 1                    
                            ## make a copy
                            if (rt > 1):

                                new_state_vals_nt = state_value_update[rt-1][tok]
                                next_state_value_array_nt[:] = new_state_vals_nt[score_gained:score_gained+score_max_plus1]
                                
                                if tok > 0:
                                    new_state_vals_t = state_value_update[rt-1][tok-1]
                                    next_state_value_array_t[:] = new_state_vals_t[score_gained:score_gained+score_max_plus1]

                            ## transit to next turn when rt=1
                            else:
                                new_state_vals_nt = optimal_value_rt3[tok]
                                next_state_value_array_nt[:] = new_state_vals_nt[score_remain:score_remain-score_max_plus1:-1]
                                
                                if tok > 0: 
                                    new_state_vals_t = optimal_value_rt3[tok-1]
                                    next_state_value_array_t[:] = new_state_vals_t[score_remain:score_remain-score_max_plus1:-1]
                                    
        
                        ## matrix product to compute all together
                        next_state_value_tensor_nt = torch.from_numpy(next_state_value_array_nt)
                        next_state_value_tensor_t = torch.from_numpy(next_state_value_array_t)
                        ## transit to next throw in the same turn when rt=3,2
                        if (rt > 1): 

                            # add one to indicate end of turn 
                            try: 
                                num_turns_tensor = torch.zeros((791,next_state_value_array_nt.shape[1]))
                            except:
                                num_turns_tensor = torch.zeros(791)
                            
                            # if we have tokens 
                            if tok > 0:  

                                # use no token probabilities for no token actions 
                                num_turns_tensor[:729] += prob_normalscore_tensor_nt[:729].matmul(next_state_value_tensor_nt)
                                # use token probabilities for token actions 
                                num_turns_tensor[729:791] += prob_normalscore_tensor_t[729:].matmul(next_state_value_tensor_t)

                            # if we don't have tokens   
                            else:
                                # add no token expectation --> index is 1 because we keep the same # of tokens       
                                num_turns_tensor = prob_normalscore_tensor_nt.matmul(next_state_value_tensor_nt)
                            
                        ## transit to next turn when rt=1
                        else:

                            # add one to indicate end of turn 
                            try: 
                                num_turns_tensor = torch.ones((791,next_state_value_array_nt.shape[1]))
                            except:
                                num_turns_tensor = torch.ones(791)
                        
                            # if we have tokens 
                            if tok > 0: 
                                # use no token probabilities for no token actions
                                num_turns_tensor[:729] += prob_normalscore_tensor_nt[:729].matmul(next_state_value_tensor_nt)
                                
                                # use token probabilities for token actions 
                                num_turns_tensor[729:791] += prob_normalscore_tensor_t[729:].matmul(next_state_value_tensor_t)
                            else: 
                                # add no token expectation  
                                num_turns_tensor += prob_normalscore_tensor_nt.matmul(next_state_value_tensor_nt)

                        ## searching
                        #temp1 = num_turns_tensor.min(axis=0) 
                        if tok == 0: 
                            temp1 = num_turns_tensor[:729].min(axis=0)
                        else: 
                            temp1 = num_turns_tensor.min(axis=0)               
                        state_action_update[rt][tok][0:state_notbust_update_index] = temp1.indices.numpy()
                        state_value_update[rt][tok][0:state_notbust_update_index] =  temp1.values.numpy() 

                    ## CASE 2: state which possibly bust.  score_state-score_gained<62 
                    if (state_notbust_len < this_throw_state_len):
                        ## combine all bust states together 
                        state_bust_len = this_throw_state_len - state_notbust_len
                        next_state_value_array_nt = np.zeros((61, state_bust_len))
                        next_state_value_array_t = np.zeros((61, state_bust_len))
                        for score_gained in range(state_notbust_len, this_throw_state_len):
                            ## skip infeasible state
                            if not fb.state_feasible_array[rt, score_gained]:
                                continue
                            score_remain = score_state - score_gained
                            #score_max = min(score_remain-2, 60)
                            score_max = score_remain-2 ## less than 60 here
                            score_max_plus1 = score_max + 1
                            score_gained_index = score_gained - state_notbust_len ## index off set
                            if (rt > 1):

                                new_state_vals_nt = state_value_update[rt-1][tok]
                                next_state_value_array_nt[0:score_max_plus1,score_gained_index] = new_state_vals_nt[score_gained:score_gained+score_max_plus1]

                                if tok > 0: 
                                    new_state_vals_t = state_value_update[rt-1][tok-1]
                                    next_state_value_array_t[0:score_max_plus1,score_gained_index] = new_state_vals_t[score_gained:score_gained+score_max_plus1]

                            ## transit to next turn when rt=1
                            else:

                                new_state_vals_nt = optimal_value_rt3[tok]
                                next_state_value_array_nt[0:score_max_plus1,score_gained_index] = new_state_vals_nt[score_remain:score_remain-score_max_plus1:-1]

                                
                                if tok > 0: 
                                    new_state_vals_t = optimal_value_rt3[tok-1]
                                    next_state_value_array_t[0:score_max_plus1,score_gained_index] = new_state_vals_t[score_remain:score_remain-score_max_plus1:-1]
                                    
                        next_state_value_tensor_nt = torch.from_numpy(next_state_value_array_nt)
                        next_state_value_tensor_t = torch.from_numpy(next_state_value_array_t)
                        
                        ## transit to next throw in the same turn when rt=3,2
                        if (rt > 1):  

                            # initialized
                            try: 
                                num_turns_tensor = torch.zeros((791,next_state_value_array_nt.shape[1]))
                            except:
                                num_turns_tensor = torch.zeros(791)
                        
                            # if we have tokens 
                            if tok > 0: 
                                # use no token probabilities for no token actions
                                num_turns_tensor[:729] += prob_normalscore_tensor_nt[:729].matmul(next_state_value_tensor_nt)
                                
                                # use token probabilities for token actions 
                                num_turns_tensor[729:791] += prob_normalscore_tensor_t[729:].matmul(next_state_value_tensor_t)
                            else: 
                                # add no token expectation  
                                num_turns_tensor += prob_normalscore_tensor_nt.matmul(next_state_value_tensor_nt)

                                            
                        ## transit to next turn when rt=1
                        else:

                            # add one to indicate end of turn 
                            try: 
                                num_turns_tensor = torch.ones((791,next_state_value_array_nt.shape[1]))
                            except:
                                num_turns_tensor = torch.ones(791)
                        
                            # if we have tokens 
                            if tok > 0: 
                                # use no token probabilities for no token actions
                                num_turns_tensor[:729] += prob_normalscore_tensor_nt[:729].matmul(next_state_value_tensor_nt)
                                
                                # use token probabilities for token actions 
                                num_turns_tensor[729:791] += prob_normalscore_tensor_t[729:].matmul(next_state_value_tensor_t)
                            else: 
                                # add no token expectation  
                                num_turns_tensor += prob_normalscore_tensor_nt.matmul(next_state_value_tensor_nt)

                            # # add one to indicate end of turn 
                            # num_turns_tensor = 1 
                            # # add no token expectation  
                            # num_turns_tensor += prob_normalscore_tensor_nt.matmul(next_state_value_tensor[0])
                            
                            # if tok > 0: 
                            #     # add token expectation 
                            #     num_turns_tensor += prob_normalscore_tensor_t.matmul(next_state_value_tensor[1])                                                             
        
                        ## consider bust/finishing for each bust state seperately 
                        num_turns_array = num_turns_tensor.numpy() 
                        ## for every score gained where you go bust                
                        for score_gained in range(state_notbust_len, this_throw_state_len):
                            ## skip infeasible state
                            if not fb.state_feasible_array[rt, score_gained]:
                                continue
                            score_remain = score_state - score_gained
                            #score_max = min(score_remain-2, 60)
                            score_max = score_remain-2 ## less than 60 here
                            score_max_plus1 = score_max + 1
                            score_gained_index = score_gained - state_notbust_len
                            
                            ## transit to the end of game
                            if (rt > 1):
                                if (score_remain == fb.score_DB):                        
                                    num_turns_array[:729,score_gained_index] += prob_DB_nt[:729]
                                    num_turns_array[729:,score_gained_index] += prob_DB_t[729:]
                                elif (score_remain <= 40 and score_remain%2==0):
                                    num_turns_array[:729,score_gained_index] += prob_doublescore_dic_nt[score_remain][:729]
                                    num_turns_array[729:,score_gained_index] += prob_doublescore_dic_t[score_remain][729:]
                                else:
                                    pass
        
                            ## transit to bust
                            if (rt==3):
                                # In this case you really are staying in the same state becuase it's the first throw of the turn
                                # Unless you go bust with the current token policy (should never happen, but need to include to avoid choosing those actions)
                                if tok > 0: 
                                    
                                    # For no token, add the probbability of staying the same with nt 
                                    num_turns_array[:729,score_gained_index] += prob_bust_dic_nt[score_max][:729]
                                    ## solve an equation other than using the policy evaluation value (s,i=3,u=0)
                                    num_turns_array[:729,score_gained_index] = num_turns_array[:729,score_gained_index] / prob_notbust_dic_nt[score_max][:729]
                                    
                                    # For token, add the probability of going to the new rt3 state (1 should already be included)
                                    # TODO check that this makes sense - not sure about the +1 here 
                                    num_turns_array[729:,score_gained_index] += prob_bust_dic_t[score_max][729:]*(1+optimal_value_rt3[tok-1,score_state])  ## 1 turn is already counted before
                                    ## solve an equation other than using the policy evaluation value (s,i=3,u=0)
                                    num_turns_array[729:,score_gained_index] = num_turns_array[729:,score_gained_index] / prob_notbust_dic_t[score_max][729:]
                                    
                                    
                                else: 
                                    # If no token, use this logic with nt 
                                    num_turns_array[:,score_gained_index] += prob_bust_dic_nt[score_max]
                                    ## solve an equation other than using the policy evaluation value (s,i=3,u=0)
                                    num_turns_array[:,score_gained_index] = num_turns_array[:,score_gained_index] / prob_notbust_dic_nt[score_max] 
                                    
                            elif (rt==2):
                                num_turns_array[:729,score_gained_index] += prob_bust_dic_nt[score_max][:729]*(1+new_value_rt3)
                                num_turns_array[729:,score_gained_index] += prob_bust_dic_t[score_max][729:]*(1+optimal_value_rt3[tok-1,score_state])
                            else:
                                # TODO check that the +1 makes sense here 
                                num_turns_array[:729,score_gained_index] += prob_bust_dic_nt[score_max][:729]*(new_value_rt3)  ## 1 turn is already counted before
                                num_turns_array[729:,score_gained_index] += prob_bust_dic_t[score_max][729:]*(1+optimal_value_rt3[tok-1,score_state])
                        
                        ## searching
                        if tok == 0: 
                            temp1 = num_turns_tensor[:729].min(axis=0)
                        else: 
                            temp1 = num_turns_tensor.min(axis=0)
                        state_action_update[rt][tok][state_notbust_len:this_throw_state_len] = temp1.indices.numpy()
                        state_value_update[rt][tok][state_notbust_len:this_throw_state_len] =  temp1.values.numpy()                
        
                    #### finish rt=1,2,3. check improvement
                    action_diff[rt][tok][:] = np.abs(state_action_update[rt][tok] - state_action[rt][tok])                                
                    value_relerror[rt] = np.abs((state_value_update[rt] - state_value[rt])/state_value_update[rt]).max()
                    state_action[rt][tok][:] = state_action_update[rt][tok][:]
                    state_value[rt][tok][:] = state_value_update[rt][tok][:]
        
                max_action_diff = max([action_diff[1].max(), action_diff[2].max(), action_diff[3].max()])
                max_value_relerror = value_relerror.max()            
                
                if (max_action_diff < 1):
                #if max_value_relerror < iteration_relerror_limit:
                    num_iteration_record[score_state] = round_index + 1
                    break


            for rt in [1,2,3]:
                state_value_update[rt][tok][fb.state_infeasible[rt]] = fb.largenumber
                state_action_update[rt][tok][fb.state_infeasible[rt]] = fb.infeasible_marker
            optimal_action_index_dic[score_state] = ft.copy_numberarray_container(state_action_update)
            optimal_value_dic[score_state] = ft.copy_numberarray_container(state_value_update, new_dtype=fb.result_float_dytpe)
            optimal_value_rt3[tok][score_state] = state_value[3][tok][0]

    #prob_scorestate_transit = {}    
    #prob_scorestate_transit =  fep.solve_policy_transit_probability(optimal_action_index_dic, prob_grid_normalscore, prob_grid_doublescore, prob_grid_bullscore)
    t2 = time.time()
    print('solve dp_turn_policyiter in {} seconds'.format(t2-t1))

    print(optimal_value_rt3)
    result_dic = {'optimal_value_dic':optimal_value_dic, 'optimal_action_index_dic':optimal_action_index_dic, 'optimal_value_rt3':optimal_value_rt3}

    return result_dic 


In [ ]:
import helpers as h
import copy
tokens = 9
max_score_state = 501
tokens_plus_one = tokens + 1

epsilon = 128


debug = []

[aiming_grid, prob_grid_normalscore_nt, prob_grid_singlescore_nt, prob_grid_doublescore_nt, prob_grid_triplescore_nt, prob_grid_bullscore_nt] = h.load_aiming_grid(name_pa, epsilon=epsilon, data_parameter_dir=data_parameter_dir, grid_version='custom_no_tokens')
[aiming_grid, prob_grid_normalscore_t, prob_grid_singlescore_t, prob_grid_doublescore_t, prob_grid_triplescore_t, prob_grid_bullscore_t] = h.load_aiming_grid('t', data_parameter_dir=data_parameter_dir, grid_version='custom_tokens')

# [aiming_grid, prob_grid_normalscore_nt, prob_grid_singlescore_nt, prob_grid_doublescore_nt, prob_grid_triplescore_nt, prob_grid_bullscore_nt] = fgag.load_aiming_grid(name_pa, data_parameter_dir=data_parameter_dir, grid_version='custom_no_tokens')
# [aiming_grid, prob_grid_normalscore_t, prob_grid_singlescore_t, prob_grid_doublescore_t, prob_grid_triplescore_t, prob_grid_bullscore_t] = fgag.load_aiming_grid(name_pa, data_parameter_dir=data_parameter_dir, grid_version='custom_tokens')

## Initialize probability objects
num_aiming_location, prob_normalscore_nt, prob_doublescore_dic_nt, prob_DB_nt, prob_bust_dic_nt, prob_notbust_dic_nt, prob_normalscore_t, prob_doublescore_dic_t, prob_DB_t, prob_bust_dic_t, prob_notbust_dic_t = init_probabilities( aiming_grid, prob_grid_normalscore_nt, prob_grid_doublescore_nt, prob_grid_bullscore_nt, prob_grid_normalscore_t, prob_grid_doublescore_t, prob_grid_bullscore_t)    

# Combined probabilities  
prob_grid_normalscore_comb = np.zeros(prob_grid_normalscore_t.shape)
prob_grid_normalscore_comb[:729,:] = prob_grid_normalscore_nt[:729,:]
prob_grid_normalscore_comb[729:] = prob_grid_normalscore_t[729:]

prob_grid_doublescore_comb = np.zeros(prob_grid_doublescore_t.shape)
prob_grid_doublescore_comb[:729,:] = prob_grid_doublescore_nt[:729,:]
prob_grid_doublescore_comb[729:] = prob_grid_doublescore_t[729:]

prob_grid_bullscore_comb = np.zeros(prob_grid_bullscore_t.shape)
prob_grid_bullscore_comb[:729,:] = prob_grid_bullscore_nt[:729,:]
prob_grid_bullscore_comb[729:] = prob_grid_bullscore_t[729:]

# Initialize joint probability notbust dictionary 
prob_notbust_dic = {}
for score_max in prob_notbust_dic_t.keys():
    prob_notbust_dic[score_max] = np.zeros(prob_notbust_dic_t[0].shape[0]) # 791 
    prob_notbust_dic[score_max][:729] += prob_notbust_dic_nt[score_max][:729] 
    prob_notbust_dic[score_max][729:] += prob_notbust_dic_t[score_max][729:]

# Intialize joint probability bust dictionary 
prob_bust_dic = {}
for score_max in prob_notbust_dic_t.keys():
    prob_bust_dic[score_max] = np.zeros(prob_bust_dic_t[0].shape[0]) # 791 
    prob_bust_dic[score_max][:729] += prob_bust_dic_nt[score_max][:729] 
    prob_bust_dic[score_max][729:] += prob_bust_dic_t[score_max][729:]


prob_normalscore_tensor_nt = torch.from_numpy(prob_normalscore_nt)
prob_normalscore_tensor_t = torch.from_numpy(prob_normalscore_t)
prob_normalscore_tensor = torch.from_numpy(prob_grid_normalscore_comb)

# prob_doublescore_dic = prob_doublescore_dic_nt
# prob_DB = prob_DB_nt
# prob_bust_dic = prob_bust_dic_nt
# prob_notbust_dic = prob_notbust_dic_nt

iteration_round_limit = 20

optimal_value_rt3 = np.zeros((tokens_plus_one,502)) #vector: optimal value for the beginning state of each turn (rt=3)
optimal_value_dic = {} ## first key: score=0,2,...,501, second key: remaining throws=3,2,1
optimal_action_index_dic = {}
num_iteration_record = np.zeros(502, dtype=np.int32)

state_len_vector = np.zeros(4, dtype=np.int32)
state_value  = [None]  ## optimal value (expected # of turns to finish the game) for each state in the current playing turn
state_action = [None]  ## aimming locations for for each state in the current playing turn
action_diff  = [None]
value_relerror = np.zeros(4)

for rt in [1,2,3]:
    ## for rt=3: possible score_gained = 0
    ## for rt=2: possible score_gained = 0,1,...,60
    ## for rt=1: possible score_gained = 0,1,...,120
    this_throw_state_len = fb.maxhitscore*(3-rt) + 1
    state_value.append(np.ones((tokens_plus_one,this_throw_state_len))*fb.largenumber)
    state_action.append(np.ones((tokens_plus_one,this_throw_state_len), np.int32)*fb.infeasible_marker)
    action_diff.append(np.ones((tokens_plus_one,this_throw_state_len)))
state_value_update = ft.copy_numberarray_container(state_value)
state_action_update = ft.copy_numberarray_container(state_action)

## use no_turn policy as the initial policy
#[noturn_optimal_value, noturn_optimal_action_index] = solve_dp_noturn(aiming_grid, prob_grid_normalscore, prob_grid_doublescore, prob_grid_bullscore)
[noturn_optimal_value, noturn_optimal_action_index] = h.solve_dp_noturn_tokens(aiming_grid, prob_grid_normalscore_nt, prob_grid_normalscore_t, tokens = tokens, prob_grid_doublescore=prob_grid_doublescore_nt, prob_grid_bullscore=prob_grid_bullscore_nt, prob_grid_doublescore_dic=None, prob_grid_doublescore_t=prob_grid_doublescore_t, prob_grid_bullscore_t=prob_grid_bullscore_t, prob_grid_doublescore_dic_t=None)

t1 = time.time()
for score_state in range(2,max_score_state+1):

    for tok in range(tokens_plus_one):

        ## initialization 
        for rt in [1,2,3]:
        
            ## for rt=3: score_gained = 0
            ## for rt=2: score_gained = 0,1,...,min(s-2,60)
            ## for rt=1: score_gained = 0,1,...,min(s-2,120)
            this_throw_state_len = min(score_state-2, fb.maxhitscore*(3-rt)) + 1
            state_len_vector[rt] = this_throw_state_len
                    
            ## initialize the starting policy: 
            ## use no_turn action in (s, i, u=0)
            ## use turn action (s-1, i, u-1) in (s, i, u!=0) if (s-1, i, u-1) is feasible state
            state_action[rt][tok][0] = noturn_optimal_action_index[tok][score_state]            
            for score_gained in range(1,this_throw_state_len):                
                if fb.state_feasible_array[rt, score_gained]:  ## if True
                    if fb.state_feasible_array[rt, score_gained-1]:
                        state_action[rt][tok][score_gained] = optimal_action_index_dic[score_state-1][rt][tok][score_gained-1]
                    else:                        
                        state_action[rt][tok][score_gained] = noturn_optimal_action_index[tok][score_state-score_gained]
                else:
                    state_action[rt][tok][score_gained] = fb.infeasible_marker
    
    # for tok in range(tokens_plus_one):

        # policy iteration
        for round_index in range(iteration_round_limit):

            ## --------------------------------------------- ##
            ##                 POLICY EVALUATION
            ## --------------------------------------------- ##
            rt = 3
            score_gained = 0
            score_max_turn = min(score_state-2, 3*fb.maxhitscore)

            # Get the transit probabilities 
            prob_turn_transit = solve_turn_transit_probability_fast_token(score_state, state_action, tok, prob_grid_normalscore_nt, prob_grid_doublescore_nt, prob_grid_bullscore_nt, prob_bust_dic_nt,prob_grid_normalscore_t, prob_grid_doublescore_t, prob_grid_bullscore_t, prob_bust_dic_t)
            
            tokens_max_turn = min(prob_turn_transit['bust'].shape[0]-1,tok)

            # Make updates below 
            prob_turn_zeroscore = prob_turn_transit['bust'][0] + prob_turn_transit['score'][0,0]

            new_value_rt3 = 1 

            # Don't use a token 
            new_value_rt3 += np.dot(prob_turn_transit['score'][0,1:], optimal_value_rt3[tok,score_state-1:score_state-score_max_turn-1:-1])
            
            # Use at least one token 
            if tok > 0: 

                # Go to subsequent score after using at least one token
                for i in range(1,tokens_max_turn+1):
                    #new_value_rt3 += np.dot(prob_turn_transit['score'][0:tokens_max_turn+1,:][i], optimal_value_rt3[tok-tokens_max_turn:tok+1,score_state:score_state-score_max_turn-1:-1][i])

                    new_value_rt3 += np.dot(np.flip(prob_turn_transit['score'][0:tokens_max_turn+1,:],axis=0)[i], optimal_value_rt3[tok-tokens_max_turn:tok+1,score_state:score_state-score_max_turn-1:-1][i])
               
                # Go bust after using at least one token 
                new_value_rt3 += np.dot(np.flip(prob_turn_transit['bust'][1:tokens_max_turn+1]) , optimal_value_rt3[tok-tokens_max_turn+1:tok+1,score_state])

            # Normalize over probability of moving to a new state 
            new_value_rt3 = new_value_rt3 / (1-prob_turn_zeroscore)
            new_value_rt3

            state_value_update[rt][tok][score_gained] = new_value_rt3
            optimal_value_rt3[tok][score_state] = new_value_rt3
            #print('evaluate rt3 value= {}'.format(new_value_rt3)

            ## --------------------------------------------- ##
            ##                 POLICY IMPROVEMENT
            ## --------------------------------------------- ## 

            for rt in [1,2,3]: 

                this_throw_state_len = state_len_vector[rt]
                state_notbust_len =  max(min(score_state-61, this_throw_state_len),0)
                token_index = min(2,tok+1)

                ## CASE 1: state which can not bust.  score_state-score_gained>=62 
                if (state_notbust_len > 0):
                    # One throw remaining and first round of iteration 
                    if (rt==1 and round_index==0):
                        ## combine all non-bust states together 
                        state_notbust_update_index = state_notbust_len                    
                        next_state_value_array_nt = np.zeros((61, state_notbust_len))   
                        next_state_value_array_t = np.zeros((61, state_notbust_len))             
                        for score_gained in range(state_notbust_len):
                            ## skip infeasible state
                            if not fb.state_feasible_array[rt, score_gained]:
                                continue
                            score_remain = score_state - score_gained
                            score_max = 60 ## always 60 here
                            score_max_plus1 = score_max + 1

                            new_state_vals_nt = optimal_value_rt3[tok]
                            next_state_value_array_nt[:,score_gained] = new_state_vals_nt[score_remain:score_remain-score_max_plus1:-1]

                            if tok > 0: 
                                new_state_vals_t = optimal_value_rt3[tok-1]
                                next_state_value_array_t[:,score_gained] = new_state_vals_t[score_remain:score_remain-score_max_plus1:-1]
                                
                    # Two throws remaining and first round of iteration 
                    elif (rt==2 and (round_index==0 or score_state<182)):
                        ## combine all non-bust states together 
                        state_notbust_update_index = state_notbust_len
                        next_state_value_array_nt = np.zeros((61, state_notbust_len))      
                        next_state_value_array_t = np.zeros((61, state_notbust_len))                      
                        for score_gained in range(state_notbust_len):
                            ## skip infeasible state
                            if not fb.state_feasible_array[rt, score_gained]:
                                continue
                            score_remain = score_state - score_gained
                            score_max = 60 ## always 60 here
                            score_max_plus1 = score_max + 1

                            new_state_vals_nt = state_value_update[rt-1][tok]
                            next_state_value_array_nt[:,score_gained] = new_state_vals_nt[score_gained:score_gained+score_max_plus1]

                            if tok > 0: 
                                new_state_vals_t = state_value_update[rt-1][tok-1]
                                next_state_value_array_t[:,score_gained] = new_state_vals_t[score_gained:score_gained+score_max_plus1]

                                
                    # Three throws remaining 
                    # OR two throws remaining past first round of iteration and score state is greater than 182 
                    # OR one throw remaining past first round of iteration  
                    else: ##(rt==1 and round_index>0) or (rt==2 and round_index>0 and score_state>=182) or (rt==3)
                        ## only update state of score_gained = 0
                        state_notbust_update_index = 1
                        next_state_value_array_nt= np.zeros((61))
                        next_state_value_array_t= np.zeros((61))
                        score_gained = 0
                        score_remain = score_state - score_gained
                        score_max = 60 ## always 60 here
                        score_max_plus1 = score_max + 1                    
                        ## make a copy
                        if (rt > 1):

                            new_state_vals_nt = state_value_update[rt-1][tok]
                            next_state_value_array_nt[:] = new_state_vals_nt[score_gained:score_gained+score_max_plus1]
                            
                            if tok > 0:
                                new_state_vals_t = state_value_update[rt-1][tok-1]
                                next_state_value_array_t[:] = new_state_vals_t[score_gained:score_gained+score_max_plus1]

                        ## transit to next turn when rt=1
                        else:
                            new_state_vals_nt = optimal_value_rt3[tok]
                            next_state_value_array_nt[:] = new_state_vals_nt[score_remain:score_remain-score_max_plus1:-1]
                            
                            if tok > 0: 
                                new_state_vals_t = optimal_value_rt3[tok-1]
                                next_state_value_array_t[:] = new_state_vals_t[score_remain:score_remain-score_max_plus1:-1]
                                
    
                    ## matrix product to compute all together
                    next_state_value_tensor_nt = torch.from_numpy(next_state_value_array_nt)
                    next_state_value_tensor_t = torch.from_numpy(next_state_value_array_t)
                    ## transit to next throw in the same turn when rt=3,2
                    if (rt > 1): 

                        # add one to indicate end of turn 
                        try: 
                            num_turns_tensor = torch.zeros((791,next_state_value_array_nt.shape[1]))
                        except:
                            num_turns_tensor = torch.zeros(791)
                        
                        # if we have tokens 
                        if tok > 0:  

                            # use no token probabilities for no token actions 
                            num_turns_tensor[:729] += prob_normalscore_tensor_nt[:729].matmul(next_state_value_tensor_nt)
                            # use token probabilities for token actions 
                            num_turns_tensor[729:791] += prob_normalscore_tensor_t[729:].matmul(next_state_value_tensor_t)

                        # if we don't have tokens   
                        else:
                            # add no token expectation --> index is 1 because we keep the same # of tokens       
                            num_turns_tensor = prob_normalscore_tensor_nt.matmul(next_state_value_tensor_nt)
                        
                    ## transit to next turn when rt=1
                    else:

                        # add one to indicate end of turn 
                        try: 
                            num_turns_tensor = torch.ones((791,next_state_value_array_nt.shape[1]))
                        except:
                            num_turns_tensor = torch.ones(791)
                    
                        # if we have tokens 
                        if tok > 0: 
                            # use no token probabilities for no token actions
                            num_turns_tensor[:729] += prob_normalscore_tensor_nt[:729].matmul(next_state_value_tensor_nt)
                            
                            # use token probabilities for token actions 
                            num_turns_tensor[729:791] += prob_normalscore_tensor_t[729:].matmul(next_state_value_tensor_t)
                        else: 
                            # add no token expectation  
                            num_turns_tensor += prob_normalscore_tensor_nt.matmul(next_state_value_tensor_nt)

                    ## searching
                    #temp1 = num_turns_tensor.min(axis=0) 
                    if tok == 0: 
                        temp1 = num_turns_tensor[:729].min(axis=0)
                    else: 
                        temp1 = num_turns_tensor.min(axis=0)               
                    state_action_update[rt][tok][0:state_notbust_update_index] = temp1.indices.numpy()
                    state_value_update[rt][tok][0:state_notbust_update_index] =  temp1.values.numpy() 

                ## CASE 2: state which possibly bust.  score_state-score_gained<62 
                if (state_notbust_len < this_throw_state_len):
                    ## combine all bust states together 
                    state_bust_len = this_throw_state_len - state_notbust_len
                    next_state_value_array_nt = np.zeros((61, state_bust_len))
                    next_state_value_array_t = np.zeros((61, state_bust_len))
                    for score_gained in range(state_notbust_len, this_throw_state_len):
                        ## skip infeasible state
                        if not fb.state_feasible_array[rt, score_gained]:
                            continue
                        score_remain = score_state - score_gained
                        #score_max = min(score_remain-2, 60)
                        score_max = score_remain-2 ## less than 60 here
                        score_max_plus1 = score_max + 1
                        score_gained_index = score_gained - state_notbust_len ## index off set
                        if (rt > 1):

                            new_state_vals_nt = state_value_update[rt-1][tok]
                            next_state_value_array_nt[0:score_max_plus1,score_gained_index] = new_state_vals_nt[score_gained:score_gained+score_max_plus1]

                            if tok > 0: 
                                new_state_vals_t = state_value_update[rt-1][tok-1]
                                next_state_value_array_t[0:score_max_plus1,score_gained_index] = new_state_vals_t[score_gained:score_gained+score_max_plus1]

                        ## transit to next turn when rt=1
                        else:

                            new_state_vals_nt = optimal_value_rt3[tok]
                            next_state_value_array_nt[0:score_max_plus1,score_gained_index] = new_state_vals_nt[score_remain:score_remain-score_max_plus1:-1]

                            
                            if tok > 0: 
                                new_state_vals_t = optimal_value_rt3[tok-1]
                                next_state_value_array_t[0:score_max_plus1,score_gained_index] = new_state_vals_t[score_remain:score_remain-score_max_plus1:-1]
                                
                    next_state_value_tensor_nt = torch.from_numpy(next_state_value_array_nt)
                    next_state_value_tensor_t = torch.from_numpy(next_state_value_array_t)
                    
                    ## transit to next throw in the same turn when rt=3,2
                    if (rt > 1):  

                        # initialized
                        try: 
                            num_turns_tensor = torch.zeros((791,next_state_value_array_nt.shape[1]))
                        except:
                            num_turns_tensor = torch.zeros(791)
                    
                        # if we have tokens 
                        if tok > 0: 
                            # use no token probabilities for no token actions
                            num_turns_tensor[:729] += prob_normalscore_tensor_nt[:729].matmul(next_state_value_tensor_nt)
                            
                            # use token probabilities for token actions 
                            num_turns_tensor[729:791] += prob_normalscore_tensor_t[729:].matmul(next_state_value_tensor_t)
                        else: 
                            # add no token expectation  
                            num_turns_tensor += prob_normalscore_tensor_nt.matmul(next_state_value_tensor_nt)

                                          
                    ## transit to next turn when rt=1
                    else:

                        # add one to indicate end of turn 
                        try: 
                            num_turns_tensor = torch.ones((791,next_state_value_array_nt.shape[1]))
                        except:
                            num_turns_tensor = torch.ones(791)
                    
                        # if we have tokens 
                        if tok > 0: 
                            # use no token probabilities for no token actions
                            num_turns_tensor[:729] += prob_normalscore_tensor_nt[:729].matmul(next_state_value_tensor_nt)
                            
                            # use token probabilities for token actions 
                            num_turns_tensor[729:791] += prob_normalscore_tensor_t[729:].matmul(next_state_value_tensor_t)
                        else: 
                            # add no token expectation  
                            num_turns_tensor += prob_normalscore_tensor_nt.matmul(next_state_value_tensor_nt)

                        # # add one to indicate end of turn 
                        # num_turns_tensor = 1 
                        # # add no token expectation  
                        # num_turns_tensor += prob_normalscore_tensor_nt.matmul(next_state_value_tensor[0])
                        
                        # if tok > 0: 
                        #     # add token expectation 
                        #     num_turns_tensor += prob_normalscore_tensor_t.matmul(next_state_value_tensor[1])                                                             
    
                    ## consider bust/finishing for each bust state seperately 
                    num_turns_array = num_turns_tensor.numpy() 
                    ## for every score gained where you go bust                
                    for score_gained in range(state_notbust_len, this_throw_state_len):
                        ## skip infeasible state
                        if not fb.state_feasible_array[rt, score_gained]:
                            continue
                        score_remain = score_state - score_gained
                        #score_max = min(score_remain-2, 60)
                        score_max = score_remain-2 ## less than 60 here
                        score_max_plus1 = score_max + 1
                        score_gained_index = score_gained - state_notbust_len
                        
                        ## transit to the end of game
                        if (rt > 1):
                            if (score_remain == fb.score_DB):                        
                                num_turns_array[:729,score_gained_index] += prob_DB_nt[:729]
                                num_turns_array[729:,score_gained_index] += prob_DB_t[729:]
                            elif (score_remain <= 40 and score_remain%2==0):
                                num_turns_array[:729,score_gained_index] += prob_doublescore_dic_nt[score_remain][:729]
                                num_turns_array[729:,score_gained_index] += prob_doublescore_dic_t[score_remain][729:]
                            else:
                                pass
    
                        ## transit to bust
                        if (rt==3):
                            # In this case you really are staying in the same state becuase it's the first throw of the turn
                            # Unless you go bust with the current token policy (should never happen, but need to include to avoid choosing those actions)
                            if tok > 0: 
                                
                                # For no token, add the probbability of staying the same with nt 
                                num_turns_array[:729,score_gained_index] += prob_bust_dic_nt[score_max][:729]
                                ## solve an equation other than using the policy evaluation value (s,i=3,u=0)
                                num_turns_array[:729,score_gained_index] = num_turns_array[:729,score_gained_index] / prob_notbust_dic_nt[score_max][:729]
                                
                                # For token, add the probability of going to the new rt3 state (1 should already be included)
                                # TODO check that this makes sense - not sure about the +1 here 
                                num_turns_array[729:,score_gained_index] += prob_bust_dic_t[score_max][729:]*(1+optimal_value_rt3[tok-1,score_state])  ## 1 turn is already counted before
                                ## solve an equation other than using the policy evaluation value (s,i=3,u=0)
                                num_turns_array[729:,score_gained_index] = num_turns_array[729:,score_gained_index] / prob_notbust_dic_t[score_max][729:]
                                
                                
                            else: 
                                # If no token, use this logic with nt 
                                num_turns_array[:,score_gained_index] += prob_bust_dic_nt[score_max]
                                ## solve an equation other than using the policy evaluation value (s,i=3,u=0)
                                num_turns_array[:,score_gained_index] = num_turns_array[:,score_gained_index] / prob_notbust_dic_nt[score_max] 
                                
                        elif (rt==2):
                            num_turns_array[:729,score_gained_index] += prob_bust_dic_nt[score_max][:729]*(1+new_value_rt3)
                            num_turns_array[729:,score_gained_index] += prob_bust_dic_t[score_max][729:]*(1+optimal_value_rt3[tok-1,score_state])
                        else:
                            # TODO check that the +1 makes sense here 
                            num_turns_array[:729,score_gained_index] += prob_bust_dic_nt[score_max][:729]*(new_value_rt3)  ## 1 turn is already counted before
                            num_turns_array[729:,score_gained_index] += prob_bust_dic_t[score_max][729:]*(1+optimal_value_rt3[tok-1,score_state])
                    
                    ## searching
                    if tok == 0: 
                        temp1 = num_turns_tensor[:729].min(axis=0)
                    else: 
                        temp1 = num_turns_tensor.min(axis=0)
                    state_action_update[rt][tok][state_notbust_len:this_throw_state_len] = temp1.indices.numpy()
                    state_value_update[rt][tok][state_notbust_len:this_throw_state_len] =  temp1.values.numpy()                
    
                #### finish rt=1,2,3. check improvement
                action_diff[rt][tok][:] = np.abs(state_action_update[rt][tok] - state_action[rt][tok])                                
                value_relerror[rt] = np.abs((state_value_update[rt] - state_value[rt])/state_value_update[rt]).max()
                state_action[rt][tok][:] = state_action_update[rt][tok][:]
                state_value[rt][tok][:] = state_value_update[rt][tok][:]
    
            max_action_diff = max([action_diff[1].max(), action_diff[2].max(), action_diff[3].max()])
            max_value_relerror = value_relerror.max()            
            
            if (max_action_diff < 1):
            #if max_value_relerror < iteration_relerror_limit:
                num_iteration_record[score_state] = round_index + 1
                break


        for rt in [1,2,3]:
            state_value_update[rt][tok][fb.state_infeasible[rt]] = fb.largenumber
            state_action_update[rt][tok][fb.state_infeasible[rt]] = fb.infeasible_marker
        optimal_action_index_dic[score_state] = ft.copy_numberarray_container(state_action_update)
        optimal_value_dic[score_state] = ft.copy_numberarray_container(state_value_update, new_dtype=fb.result_float_dytpe)
        optimal_value_rt3[tok][score_state] = state_value[3][tok][0]

        # if tok ==0: 
        #     print(optimal_value_rt3)


In [1239]:
parent_probability = np.ones((1,1))

# Establish number of tokens that must be considered in turn  
max_tokens_in_turn = min(available_tokens,3) # either 3 (use token for every throw in turn) or the total number of available tokens if < 3 
max_tokens_plus_one = max_tokens_in_turn + 1 # used for indexing purposes

# Initialize data structures to store final probabilities
result_dict = {}
result_dict['finish'] = 0 # probability of finishing the game
result_dict['bust'] = np.zeros(max_tokens_plus_one) # probability of going bust while using n tokens
prob_transit_len = min(score_state-2, fb.maxhitscore*(3)) + 1 # length of feasible transition array 
result_dict['score'] = np.zeros((max_tokens_plus_one,prob_transit_len)) # matrix of TPs; prob of transitioning to [t,s] where t is tokens used and s is score state


for rt in [3,2,1]:

    # Max score gained in turn; bounded by score state if you can go bust. Otherwise will be 60,120,180 for rt=3,2,1 respectively. 
    max_score_gained = min(score_state-2, fb.maxhitscore*(4-rt))

    # Add a one for indexing purposes 
    max_score_gained_index = max_score_gained + 1

    # Max score gained by throw 
    max_throw_score = min(score_state-2, fb.maxhitscore)
    max_throw_score_index = max_throw_score + 1

    # Token sizing 
    token_index = min(available_tokens,4-rt) + 1

    # Initialize child probability object which will contain each state that can be transitioned to, with value equal to transition prob
    child_probability = np.zeros((token_index,max_score_gained_index))

    # this fills in the parent probability 
    #prob_normalscore_transit = prob_normalscore[state_action[rt][0:this_throw_state_len]]*prob_this_throw_state.reshape((this_throw_state_len,1))
    
    for tokens_used in range(parent_probability.shape[0]):

        for score_gained in range(parent_probability.shape[1]):

            ## skip infeasible state
            if not fb.state_feasible_array[rt, score_gained]:
                continue   

            ## skip if zero probability of being in parent state in the first place to save time
            if parent_probability[tokens_used][score_gained] < 0.000000000000001:
                continue 

            ## get policy action at this state
            remaining_tokens = available_tokens-tokens_used
            action_index = state_action[rt][remaining_tokens][score_gained]
            is_token = (action_index >= 729)
            bool_token = 1 if is_token else 0 

            if ((is_token)&((max_tokens_in_turn-tokens_used)>0)): 
                prob_normalscore = prob_grid_normalscore_t
                prob_bullscore = prob_grid_bullscore_t
                prob_doublescore = prob_grid_doublescore_t
                prob_bust_dic = prob_bust_dic_t
            else: 
                prob_normalscore = prob_grid_normalscore_nt
                prob_bullscore = prob_grid_bullscore_nt
                prob_doublescore = prob_grid_doublescore_nt
                prob_bust_dic = prob_bust_dic_nt

            # Probabilitiy of being in this state of the turn (i.e. specific number of tokens used and score gained) given policy
            prob_this_state = parent_probability[tokens_used][score_gained]

            # Probability of every outcome of current throw (2x61); either 0 or 1 tokens used, score_gained of 0 to 60
            throw_transit_probability = np.zeros((min(remaining_tokens+1,2),max_throw_score_index))

            # Add transition probabilities depending on whether a token was used or not; multiply by probability of this state 
            if available_tokens ==0: 
                throw_transit_probability += prob_normalscore[action_index][:max_throw_score_index] * prob_this_state 
            else: 
                throw_transit_probability[bool_token] += prob_normalscore[action_index][:max_throw_score_index] * prob_this_state 
                # print("IS TOKEN")
                # print(is_token)
                # print('update multiplication')
                # print(prob_normalscore[action_index][:max_throw_score_index] * prob_this_state )
                # print(throw_transit_probability)
                # print(throw_transit_probability[0])
                # print(throw_transit_probability[bool_token])

            # Get the largest score that can be made without busting 
            score_remain = score_state - score_gained
            score_max = min(score_remain-2, 60)
            score_max_plus1 = score_max + 1
        
            ## Transit to next throw or turn with normal scores    
            if available_tokens ==0: 
                child_probability[0,score_gained:score_gained+score_max_plus1] += throw_transit_probability[0,0:score_max_plus1]
            else:   
                # print('rt',rt,'-','score state',score_state,'-','score gained',score_gained,'-','tokens used',tokens_used) 
                # print('child shape',child_probability[tokens_used:tokens_used+2,score_gained:score_gained+score_max_plus1].shape)
                # print('transit shape',throw_transit_probability[:,0:score_max_plus1].shape)     
                child_probability[tokens_used:tokens_used+2,score_gained:score_gained+score_max_plus1] += throw_transit_probability[:,0:score_max_plus1]
            
            ## game can not bust or end when score_max = 60, i.e.,  prob_notbust = 1
            if (score_max < 60):
                ## transit to the end of game
                if (score_remain == fb.score_DB):
                    result_dict['finish'] += prob_bullscore[action_index, 1]*prob_this_state
                elif (score_remain <= 40 and score_remain%2==0):
                    doublescore_index = (score_remain//2) - 1
                    result_dict['finish'] += prob_doublescore[action_index, doublescore_index]*prob_this_state
                else:
                    pass

                #transit to bust
                result_dict['bust'][tokens_used+bool_token] += prob_bust_dic[score_max][action_index]*prob_this_state

            # if child_probability.sum()==0 and score_state > 2:#available_tokens > 0 and score_state == 59: 
            #     print('------------------------------------------------------------------------------------')
            #     print('score',score_state,'start tokens',available_tokens,'gained',score_gained,'tokens used',tokens_used)
            #     print('throw transit probs',throw_transit_probability)
            #     print('throw transit update vals',prob_normalscore[action_index][:max_throw_score_index])
            #     print('prob this state',prob_this_state)
            #     print('rt',rt)
            #     print('parent',parent_probability)
            #     print('child',child_probability)
            #     print('finish',result_dict['finish'])
            #     print('bust',result_dict['bust'])
            #     return -1

    parent_probability = child_probability
    #print(parent_probability)

result_dict['score'] = parent_probability




IndexError: index 1 is out of bounds for axis 0 with size 1

In [1252]:
#result_dict['bust'][tokens_used+bool_token]
state_action
imdp.a_list[40]
#noturn_optimal_action_index[0][:5]

'SO1-c-m'

In [1040]:
#num_turns_tensor += prob_normalscore_tensor_t.matmul(next_state_value_tensor[1])

prob_normalscore_tensor_nt.shape
#next_state_value_tensor_nt.shape
prob_normalscore_tensor_nt.matmul(next_state_value_tensor_nt)
prob_normalscore_tensor_nt.matmul(next_state_value_tensor_nt)

tensor([[1.4251],
        [1.4598],
        [1.4668],
        ...,
        [1.7005],
        [1.7005],
        [1.7005]], dtype=torch.float64)

In [1103]:
optimal_value_rt3[0:2,70:80]
optimal_value_rt3

array([[0.    , 0.    , 1.4381, ..., 0.    , 0.    , 0.    ],
       [0.    , 0.    , 1.    , ..., 0.    , 0.    , 0.    ],
       [0.    , 0.    , 1.    , ..., 0.    , 0.    , 0.    ]])

In [949]:
#new_value_rt3 += np.dot(np.flip(prob_turn_transit['score'][0:tokens_max_turn+1,:],axis=0)[i], optimal_value_rt3[tok-tokens_max_turn:tok+1,score_state:score_state-score_max_turn-1:-1][i])

np.flip(prob_turn_transit['score'][0:tokens_max_turn+1,:],axis=0).sum(axis=1)[i]
#optimal_value_rt3[tok-tokens_max_turn:tok+1,score_state:score_state-score_max_turn-1:-1][i]

array([0., 1., 0., 0.])

In [874]:
# Baseline turn results --> [0.     0.     1.4381 ... 5.0818 5.0845 5.0965]
optimal_value_rt3

array([[0.    , 0.    , 1.4381, ..., 0.    , 0.    , 0.    ],
       [0.    , 0.    , 4.0386, ..., 0.    , 0.    , 0.    ],
       [0.    , 0.    , 5.    , ..., 0.    , 0.    , 0.    ]])

In [875]:
optimal_value_rt3[:,:5]

array([[ 0.    ,  0.    ,  1.4381,  1.5663,  1.2234],
       [ 0.    ,  0.    ,  4.0386,  5.8942,  3.5624],
       [ 0.    ,  0.    ,  5.    , 10.813 ,  6.3303]])

In [802]:
num_iteration_record[:]

array([0, 0, 1, ..., 2, 2, 2], dtype=int32)

In [840]:
optimal_value_dic

{2: [None,
  array([[   1.7498, 1000.    , 1000.    , ..., 1000.    , 1000.    , 1000.    ],
         [   2.1899, 1000.    , 1000.    , ..., 1000.    , 1000.    , 1000.    ],
         [   2.    , 1000.    , 1000.    , ..., 1000.    , 1000.    , 1000.    ]]),
  array([[   1.5265, 1000.    , 1000.    , 1000.    , 1000.    , 1000.    , 1000.    , 1000.    , 1000.    , 1000.    , 1000.    , 1000.    , 1000.    , 1000.    , 1000.    , 1000.    , 1000.    , 1000.    , 1000.    , 1000.    , 1000.    , 1000.    , 1000.    , 1000.    , 1000.    , 1000.    ,
          1000.    , 1000.    , 1000.    , 1000.    , 1000.    , 1000.    , 1000.    , 1000.    , 1000.    , 1000.    , 1000.    , 1000.    , 1000.    , 1000.    , 1000.    , 1000.    , 1000.    , 1000.    , 1000.    , 1000.    , 1000.    , 1000.    , 1000.    , 1000.    , 1000.    , 1000.    ,
          1000.    , 1000.    , 1000.    , 1000.    , 1000.    , 1000.    , 1000.    , 1000.    , 1000.    ],
         [   3.1899, 1000.    , 1000.  

In [811]:
optimal_action_index_dic[3]

[None,
 array([[ 40,  55,  -1, ...,  -1,  -1,  -1],
        [730,  56,  -1, ...,  -1,  -1,  -1],
        [730, 752,  -1, ...,  -1,  -1,  -1]], dtype=int32),
 array([[ 40,  54,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
          -1,  -1,  -1],
        [153,  56,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
          -1,  -1,  -1],
        [153, 752,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  

In [755]:
# Make updates below 
prob_turn_zeroscore = prob_turn_transit['bust'][0] + prob_turn_transit['score'][0,0]

new_value_rt3 = 1 

# # Don't use a token 
new_value_rt3 += np.dot(prob_turn_transit['score'][0,1:], optimal_value_rt3[tok,score_state-1:score_state-score_max_turn-1:-1])
new_value_rt3
# # Use at least one token 
if tok > 0: 

    # Go to subsequent score after using at least one token
    for i in range(tokens_max_turn+1):
        new_value_rt3 += np.dot(np.flip(prob_turn_transit['score'][0:tokens_max_turn+1,:],axis=0)[i], optimal_value_rt3[tok-tokens_max_turn:tok+1,score_state:score_state-score_max_turn-1:-1][i])
    # Go bust after using at least one token 
    new_value_rt3 += np.dot(prob_turn_transit['bust'][1:tokens_max_turn+1] , optimal_value_rt3[tok:tok-tokens_max_turn:-1,score_state])

# # Normalize over probability of moving to a new state 
new_value_rt3 = new_value_rt3 / (1-prob_turn_zeroscore)
new_value_rt3

1


1.0

In [749]:
prob_turn_transit
np.flip(prob_turn_transit['score'][0:tokens_max_turn+1,:],axis=0)
optimal_value_rt3[tok-tokens_max_turn:tok+1,score_state:score_state-score_max_turn-1:-1]

prob_turn_transit['bust'][1:tokens_max_turn+1]
optimal_value_rt3[tok:tok-tokens_max_turn:-1,score_state]


array([39.004,  3.   ,  4.653])

In [655]:
next_state_value_array[:1,:].shape
np.flip(new_state_vals[:,score_gained:score_gained+score_max_plus1])#.shape#,score_gained:score_gained+score_max_plus1:-1]
#new_state_vals[:,score_gained:score_gained+score_max_plus1]
new_state_vals[:,62]#,score_remain:score_remain-score_max_plus1]
score_remain
score_max_plus1

61

In [627]:
prob_grid_normalscore = prob_grid_normalscore_nt
prob_grid_doublescore = prob_grid_doublescore_nt
prob_grid_bullscore = prob_grid_bullscore_nt

## aiming_grid
num_aiming_location = aiming_grid.shape[0]
prob_normalscore = prob_grid_normalscore
prob_doublescore_dic = {}
for doublescore_index in range(20):
    doublescore = 2*(doublescore_index+1)
    prob_doublescore_dic[doublescore] = np.array(prob_grid_doublescore[:,doublescore_index])
prob_DB = np.array(prob_grid_bullscore[:,1])

## the probability of not bust for each action given score_max=i (score_remain=i+2)
prob_bust_dic = {}
prob_notbust_dic = {}
for score_max in range(60):    
    ## transit to next throw or turn
    prob_notbust = prob_grid_normalscore[:,0:score_max+1].sum(axis=1)
    ## transit to the end of game
    score_remain = score_max + 2
    if (score_remain == fb.score_DB):
        prob_notbust += prob_DB
    elif (score_remain <= 40 and score_remain%2==0):
        prob_notbust += prob_doublescore_dic[score_remain]
    ##
    prob_notbust = np.minimum(np.maximum(prob_notbust, 0),1)
    prob_notbust_dic[score_max] = prob_notbust
    prob_bust_dic[score_max] = 1 - prob_notbust_dic[score_max]

prob_normalscore_tensor = torch.from_numpy(prob_normalscore)

iteration_round_limit = 20
iteration_relerror_limit = 10**-9

#### state space example of (SB=25 DB=50) ####
## rt: the number of remaining throws in a turn
## state_infeasible_rt2 = [23, 29, 31, 35, 37, 41, 43, 44, 46, 47, 49, 52, 53, 55, 56, 58, 59]
## state_infeasible_rt1 = [103, 106, 109, 112, 113, 115, 116, 118, 119]    
    
optimal_value_rt3 = np.zeros(502) #vector: optimal value for the beginning state of each turn (rt=3)
optimal_value_dic = {} ## first key: score=0,2,...,501, second key: remaining throws=3,2,1
optimal_action_index_dic = {}
num_iteration_record = np.zeros(502, dtype=np.int32)

state_len_vector = np.zeros(4, dtype=np.int32)
state_value  = [None]  ## optimal value (expected # of turns to finish the game) for each state in the current playing turn
state_action = [None]  ## aimming locations for for each state in the current playing turn
action_diff  = [None]
value_relerror = np.zeros(4)
for rt in [1,2,3]:
    ## for rt=3: possible score_gained = 0
    ## for rt=2: possible score_gained = 0,1,...,60
    ## for rt=1: possible score_gained = 0,1,...,120
    this_throw_state_len = fb.maxhitscore*(3-rt) + 1
    state_value.append(np.ones(this_throw_state_len)*fb.largenumber)
    state_action.append(np.ones(this_throw_state_len, np.int32)*fb.infeasible_marker)
    action_diff.append(np.ones(this_throw_state_len))
state_value_update = ft.copy_numberarray_container(state_value)
state_action_update = ft.copy_numberarray_container(state_action)

## use no_turn policy as the initial policy
[noturn_optimal_value, noturn_optimal_action_index] = solve_dp_noturn(aiming_grid, prob_grid_normalscore, prob_grid_doublescore, prob_grid_bullscore)

t1 = time.time()
for score_state in range(2, 502):
    #print('#### solve_dp_turn score_state={} ####'.format(score_state))    
    
    ## initialization 
    for rt in [1,2,3]:
        ## for rt=3: score_gained = 0
        ## for rt=2: score_gained = 0,1,...,min(s-2,60)
        ## for rt=1: score_gained = 0,1,...,min(s-2,120)
        this_throw_state_len = min(score_state-2, fb.maxhitscore*(3-rt)) + 1
        state_len_vector[rt] = this_throw_state_len
                
        ## initialize the starting policy: 
        ## use no_turn action in (s, i, u=0)
        ## use turn action (s-1, i, u-1) in (s, i, u!=0) if (s-1, i, u-1) is feasible state
        state_action[rt][0] = noturn_optimal_action_index[score_state]            
        for score_gained in range(1,this_throw_state_len):                
            if fb.state_feasible_array[rt, score_gained]:  ## if True
                if fb.state_feasible_array[rt, score_gained-1]:
                    state_action[rt][score_gained] = optimal_action_index_dic[score_state-1][rt][score_gained-1]
                else:                        
                    state_action[rt][score_gained] = noturn_optimal_action_index[score_state-score_gained]
            else:
                state_action[rt][score_gained] = fb.infeasible_marker

    ## policy iteration
    for round_index in range(iteration_round_limit):

        ## policy evaluation
        rt = 3
        score_gained = 0
        score_max_turn = min(score_state-2, 3*fb.maxhitscore)
        prob_turn_transit = fep.solve_turn_transit_probability_fast(score_state, state_action, prob_grid_normalscore, prob_grid_doublescore, prob_grid_bullscore, prob_bust_dic)        
        prob_turn_zeroscore = prob_turn_transit['bust'] + prob_turn_transit['score'][0]
        new_value_rt3 = (1 + np.dot(prob_turn_transit['score'][1:], optimal_value_rt3[score_state-1:score_state-score_max_turn-1:-1])) / (1-prob_turn_zeroscore)
        state_value_update[rt][score_gained] = new_value_rt3
        optimal_value_rt3[score_state] = new_value_rt3
        #print('evaluate rt3 value= {}'.format(new_value_rt3)

        # ## policy improvement
        # for rt in [1,2,3]:            
        #     this_throw_state_len = state_len_vector[rt]
            
        #     ## state which can not bust.  score_state-score_gained>=62 
        #     state_notbust_len =  max(min(score_state-61, this_throw_state_len),0)
        #     if (state_notbust_len > 0):
        #         if (rt==1 and round_index==0):
        #             ## combine all non-bust states together 
        #             state_notbust_update_index = state_notbust_len                    
        #             next_state_value_array = np.zeros((61, state_notbust_len))                    
        #             for score_gained in range(state_notbust_len):
        #                 ## skip infeasible state
        #                 if not fb.state_feasible_array[rt, score_gained]:
        #                     continue
        #                 score_remain = score_state - score_gained
        #                 score_max = 60 ## always 60 here
        #                 score_max_plus1 = score_max + 1
        #                 next_state_value_array[:,score_gained] = optimal_value_rt3[score_remain:score_remain-score_max_plus1:-1]
        #         elif (rt==2 and (round_index==0 or score_state<182)):
        #             ## combine all non-bust states together 
        #             state_notbust_update_index = state_notbust_len
        #             next_state_value_array = np.zeros((61, state_notbust_len))                    
        #             for score_gained in range(state_notbust_len):
        #                 ## skip infeasible state
        #                 if not fb.state_feasible_array[rt, score_gained]:
        #                     continue
        #                 score_remain = score_state - score_gained
        #                 score_max = 60 ## always 60 here
        #                 score_max_plus1 = score_max + 1
        #                 next_state_value_array[:,score_gained] = state_value_update[rt-1][score_gained:score_gained+score_max_plus1]
        #         else: ##(rt==1 and round_index>0) or (rt==2 and round_index>0 and score_state>=182) or (rt==3)
        #             ## only update state of score_gained = 0
        #             state_notbust_update_index = 1
        #             next_state_value_array = np.zeros(61)
        #             score_gained = 0
        #             score_remain = score_state - score_gained
        #             score_max = 60 ## always 60 here
        #             score_max_plus1 = score_max + 1                    
        #             ## make a copy
        #             if (rt > 1):
        #                 next_state_value_array[:] = state_value_update[rt-1][score_gained:score_gained+score_max_plus1]
        #             ## transit to next turn when rt=1
        #             else:
        #                 next_state_value_array[:] = optimal_value_rt3[score_remain:score_remain-score_max_plus1:-1]

        #         ## matrix product to compute all together
        #         next_state_value_tensor = torch.from_numpy(next_state_value_array)
        #         ## transit to next throw in the same turn when rt=3,2
        #         if (rt > 1):                    
        #             num_turns_tensor = prob_normalscore_tensor.matmul(next_state_value_tensor)
        #         ## transit to next turn when rt=1
        #         else:
        #             num_turns_tensor = 1 + prob_normalscore_tensor.matmul(next_state_value_tensor)

        #         ## searching
        #         temp1 = num_turns_tensor.min(axis=0)                
        #         state_action_update[rt][0:state_notbust_update_index] = temp1.indices.numpy()
        #         state_value_update[rt][0:state_notbust_update_index] =  temp1.values.numpy()                
            
        #     ## state which possibly bust.  score_state-score_gained<62 
        #     if (state_notbust_len < this_throw_state_len):
        #         ## combine all bust states together 
        #         state_bust_len = this_throw_state_len - state_notbust_len
        #         next_state_value_array = np.zeros((61, state_bust_len))
        #         for score_gained in range(state_notbust_len, this_throw_state_len):
        #             ## skip infeasible state
        #             if not fb.state_feasible_array[rt, score_gained]:
        #                 continue
        #             score_remain = score_state - score_gained
        #             #score_max = min(score_remain-2, 60)
        #             score_max = score_remain-2 ## less than 60 here
        #             score_max_plus1 = score_max + 1
        #             score_gained_index = score_gained - state_notbust_len ## index off set
        #             if (rt > 1):
        #                 next_state_value_array[0:score_max_plus1,score_gained_index] = state_value_update[rt-1][score_gained:score_gained+score_max_plus1]
        #             ## transit to next turn when rt=1
        #             else:
        #                 next_state_value_array[0:score_max_plus1,score_gained_index] = optimal_value_rt3[score_remain:score_remain-score_max_plus1:-1]
                
        #         next_state_value_tensor = torch.from_numpy(next_state_value_array)
        #         ## transit to next throw in the same turn when rt=3,2
        #         if (rt > 1):                    
        #             num_turns_tensor = prob_normalscore_tensor.matmul(next_state_value_tensor)
        #         ## transit to next turn when rt=1
        #         else:
        #             num_turns_tensor = 1 + prob_normalscore_tensor.matmul(next_state_value_tensor)                                                               

        #         ## consider bust/finishing for each bust state seperately 
        #         num_turns_array = num_turns_tensor.numpy()                
        #         for score_gained in range(state_notbust_len, this_throw_state_len):
        #             ## skip infeasible state
        #             if not fb.state_feasible_array[rt, score_gained]:
        #                 continue
        #             score_remain = score_state - score_gained
        #             #score_max = min(score_remain-2, 60)
        #             score_max = score_remain-2 ## less than 60 here
        #             score_max_plus1 = score_max + 1
        #             score_gained_index = score_gained - state_notbust_len

        #             ## transit to the end of game
        #             if (rt > 1):
        #                 if (score_remain == fb.score_DB):                        
        #                     num_turns_array[:,score_gained_index] += prob_DB
        #                 elif (score_remain <= 40 and score_remain%2==0):
        #                     num_turns_array[:,score_gained_index] += prob_doublescore_dic[score_remain]
        #                 else:
        #                     pass

        #             ## transit to bust
        #             if (rt==3):
        #                 num_turns_array[:,score_gained_index] += prob_bust_dic[score_max]
        #                 ## solve an equation other than using the policy evaluation value (s,i=3,u=0)
        #                 num_turns_array[:,score_gained_index] = num_turns_array[:,score_gained_index] / prob_notbust_dic[score_max] 
        #             elif (rt==2):
        #                 num_turns_array[:,score_gained_index] += prob_bust_dic[score_max]*(1+new_value_rt3)
        #             else:
        #                 num_turns_array[:,score_gained_index] += prob_bust_dic[score_max]*(new_value_rt3)  ## 1 turn is already counted before

        #         ## searching
        #         temp1 = num_turns_tensor.min(axis=0)
        #         state_action_update[rt][state_notbust_len:this_throw_state_len] = temp1.indices.numpy()
        #         state_value_update[rt][state_notbust_len:this_throw_state_len] =  temp1.values.numpy()                

        #     #### finish rt=1,2,3. check improvement
        #     action_diff[rt][:] = np.abs(state_action_update[rt] - state_action[rt])                                
        #     value_relerror[rt] = np.abs((state_value_update[rt] - state_value[rt])/state_value_update[rt]).max()
        #     state_action[rt][:] = state_action_update[rt][:]
        #     state_value[rt][:] = state_value_update[rt][:]

        # max_action_diff = max([action_diff[1].max(), action_diff[2].max(), action_diff[3].max()])
        # max_value_relerror = value_relerror.max()            
        
        # if (max_action_diff < 1):
        # #if max_value_relerror < iteration_relerror_limit:
        #     num_iteration_record[score_state] = round_index + 1
        #     break

    for rt in [1,2,3]:
        state_value_update[rt][fb.state_infeasible[rt]] = fb.largenumber
        state_action_update[rt][fb.state_infeasible[rt]] = fb.infeasible_marker
    optimal_action_index_dic[score_state] = ft.copy_numberarray_container(state_action_update)
    optimal_value_dic[score_state] = ft.copy_numberarray_container(state_value_update, new_dtype=fb.result_float_dytpe)
    optimal_value_rt3[score_state] = state_value[3][0]
    ## done:V(s,i=3/2/1,u)

/var/folders/jj/jcq76kp53_zchkx8b5cvhdsh0000gn/T/ipykernel_964/78220377.py:59: RuntimeWarning: divide by zero encountered in divide
  num_tothrow = num_tothrow / prob_otherstate
/var/folders/jj/jcq76kp53_zchkx8b5cvhdsh0000gn/T/ipykernel_964/78220377.py:59: RuntimeWarning: overflow encountered in divide
  num_tothrow = num_tothrow / prob_otherstate


In [628]:
optimal_value_rt3

array([   0.,    0., 1000., ..., 1000., 1000., 1000.])

In [630]:
optimal_value_rt3

array([[   0.,    0., 1000., ..., 1000., 1000., 1000.]])

In [621]:
imdp.a_list[:5]
imdp.a_list[-5:]
imdp.a_list[728:730] # :729, 729:
prob_bust_dic

{0: array([1., 1., 1., ..., 1., 1., 1.]),
 1: array([0.9611, 0.7815, 0.7229, ..., 1.    , 1.    , 1.    ]),
 2: array([0.9611, 0.7815, 0.7229, ..., 1.    , 1.    , 1.    ]),
 3: array([0.9577, 0.7815, 0.7229, ..., 1.    , 1.    , 1.    ]),
 4: array([0.9577, 0.7815, 0.7228, ..., 1.    , 1.    , 1.    ]),
 5: array([0.8981, 0.5099, 0.4768, ..., 1.    , 1.    , 1.    ]),
 6: array([0.8981, 0.5099, 0.4768, ..., 1.    , 1.    , 1.    ]),
 7: array([0.8981, 0.5099, 0.4768, ..., 1.    , 1.    , 1.    ]),
 8: array([0.8981, 0.5099, 0.4768, ..., 1.    , 1.    , 1.    ]),
 9: array([0.8981, 0.5099, 0.476 , ..., 1.    , 1.    , 1.    ]),
 10: array([0.8981, 0.5099, 0.476 , ..., 1.    , 1.    , 1.    ]),
 11: array([0.8981, 0.5099, 0.476 , ..., 1.    , 1.    , 1.    ]),
 12: array([0.898 , 0.5075, 0.443 , ..., 1.    , 1.    , 1.    ]),
 13: array([0.898 , 0.5075, 0.443 , ..., 1.    , 1.    , 1.    ]),
 14: array([0.898 , 0.5075, 0.4423, ..., 1.    , 1.    , 1.    ]),
 15: array([0.898 , 0.5075, 0

In [619]:
prob_bust_dic = {}

for score_max in prob_notbust_dic_t.keys():
    prob_bust_dic[score_max] = np.zeros(prob_bust_dic_t[0].shape[0]) # 791 
    prob_bust_dic[score_max][:729] += prob_bust_dic_nt[score_max][:729] 
    prob_bust_dic[score_max][729:] += prob_bust_dic_t[score_max][729:]

prob_bust_dic



{0: array([1., 1., 1., ..., 1., 1., 1.]),
 1: array([0.9611, 0.7815, 0.7229, ..., 1.    , 1.    , 1.    ]),
 2: array([0.9611, 0.7815, 0.7229, ..., 1.    , 1.    , 1.    ]),
 3: array([0.9577, 0.7815, 0.7229, ..., 1.    , 1.    , 1.    ]),
 4: array([0.9577, 0.7815, 0.7228, ..., 1.    , 1.    , 1.    ]),
 5: array([0.8981, 0.5099, 0.4768, ..., 1.    , 1.    , 1.    ]),
 6: array([0.8981, 0.5099, 0.4768, ..., 1.    , 1.    , 1.    ]),
 7: array([0.8981, 0.5099, 0.4768, ..., 1.    , 1.    , 1.    ]),
 8: array([0.8981, 0.5099, 0.4768, ..., 1.    , 1.    , 1.    ]),
 9: array([0.8981, 0.5099, 0.476 , ..., 1.    , 1.    , 1.    ]),
 10: array([0.8981, 0.5099, 0.476 , ..., 1.    , 1.    , 1.    ]),
 11: array([0.8981, 0.5099, 0.476 , ..., 1.    , 1.    , 1.    ]),
 12: array([0.898 , 0.5075, 0.443 , ..., 1.    , 1.    , 1.    ]),
 13: array([0.898 , 0.5075, 0.443 , ..., 1.    , 1.    , 1.    ]),
 14: array([0.898 , 0.5075, 0.4423, ..., 1.    , 1.    , 1.    ]),
 15: array([0.898 , 0.5075, 0

In [526]:
tok=1 
new_state_vals = optimal_value_rt3[tok-1:token_index+1]
new_state_vals = np.flip(new_state_vals,axis=0)


array([[   0.    ,    0.    , 1000.    , ...,    0.    ,    0.    ,    0.    ],
       [   0.    ,    0.    ,    1.6622, ...,    0.    ,    0.    ,    0.    ]])

In [622]:
def solve_turn_transit_probability_fast_token(score_state, state_action, available_tokens, prob_grid_normalscore_nt, prob_grid_doublescore_nt, prob_grid_bullscore_nt, prob_bust_dic_nt,prob_grid_normalscore_t, prob_grid_doublescore_t, prob_grid_bullscore_t, prob_bust_dic_t):

    parent_probability = np.ones((1,1))

    # Establish number of tokens that must be considered in turn  
    max_tokens_in_turn = min(available_tokens,3) # either 3 (use token for every throw in turn) or the total number of available tokens if < 3 
    max_tokens_plus_one = max_tokens_in_turn + 1 # used for indexing purposes

    # Initialize data structures to store final probabilities
    result_dict = {}
    result_dict['finish'] = 0 # probability of finishing the game
    result_dict['bust'] = np.zeros(max_tokens_plus_one) # probability of going bust while using n tokens
    prob_transit_len = min(score_state-2, fb.maxhitscore*(3)) + 1 # length of feasible transition array 
    result_dict['score'] = np.zeros((max_tokens_plus_one,prob_transit_len)) # matrix of TPs; prob of transitioning to [t,s] where t is tokens used and s is score state


    for rt in [3,2,1]:

        # Max score gained in turn; bounded by score state if you can go bust. Otherwise will be 60,120,180 for rt=3,2,1 respectively. 
        max_score_gained = min(score_state-2, fb.maxhitscore*(4-rt))

        # Add a one for indexing purposes 
        max_score_gained_index = max_score_gained + 1

        # Max score gained by throw 
        max_throw_score = min(score_state-2, fb.maxhitscore)
        max_throw_score_index = max_throw_score + 1

        # Token sizing 
        token_index = min(available_tokens,4-rt) + 1

        # Initialize child probability object which will contain each state that can be transitioned to, with value equal to transition prob
        child_probability = np.zeros((token_index,max_score_gained_index))

        # this fills in the parent probability 
        #prob_normalscore_transit = prob_normalscore[state_action[rt][0:this_throw_state_len]]*prob_this_throw_state.reshape((this_throw_state_len,1))
        
        for tokens_used in range(parent_probability.shape[0]):

            for score_gained in range(parent_probability.shape[1]):

                ## skip infeasible state
                if not fb.state_feasible_array[rt, score_gained]:
                    continue   

                ## skip if zero probability of being in parent state in the first place to save time
                if parent_probability[tokens_used][score_gained] < 0.000000000000001:
                    continue 

                ## get policy action at this state
                remaining_tokens = available_tokens-tokens_used
                action_index = state_action[rt][remaining_tokens][score_gained]
                is_token = (action_index >= 729)
                bool_token = 1 if is_token else 0 

                if ((is_token)&((max_tokens_in_turn-tokens_used)>0)): 
                    prob_normalscore = prob_grid_normalscore_t
                    prob_bullscore = prob_grid_bullscore_t
                    prob_doublescore = prob_grid_doublescore_t
                    prob_bust_dic = prob_bust_dic_t
                else: 
                    prob_normalscore = prob_grid_normalscore_nt
                    prob_bullscore = prob_grid_bullscore_nt
                    prob_doublescore = prob_grid_doublescore_nt
                    prob_bust_dic = prob_bust_dic_nt

                # Probabilitiy of being in this state of the turn (i.e. specific number of tokens used and score gained) given policy
                prob_this_state = parent_probability[tokens_used][score_gained]

                # Probability of every outcome of current throw (2x61); either 0 or 1 tokens used, score_gained of 0 to 60
                throw_transit_probability = np.zeros((min(remaining_tokens+1,2),max_throw_score_index))

                # Add transition probabilities depending on whether a token was used or not; multiply by probability of this state 
                if available_tokens ==0: 
                    throw_transit_probability += prob_normalscore[action_index][:max_throw_score_index] * prob_this_state 
                else: 
                    throw_transit_probability[bool_token] += prob_normalscore[action_index][:max_throw_score_index] * prob_this_state 
                    # print("IS TOKEN")
                    # print(is_token)
                    # print('update multiplication')
                    # print(prob_normalscore[action_index][:max_throw_score_index] * prob_this_state )
                    # print(throw_transit_probability)
                    # print(throw_transit_probability[0])
                    # print(throw_transit_probability[bool_token])

                # Get the largest score that can be made without busting 
                score_remain = score_state - score_gained
                score_max = min(score_remain-2, 60)
                score_max_plus1 = score_max + 1
            
                ## Transit to next throw or turn with normal scores    
                if available_tokens ==0: 
                    child_probability[0,score_gained:score_gained+score_max_plus1] += throw_transit_probability[0,0:score_max_plus1]
                else:   
                    # print('rt',rt,'-','score state',score_state,'-','score gained',score_gained,'-','tokens used',tokens_used) 
                    # print('child shape',child_probability[tokens_used:tokens_used+2,score_gained:score_gained+score_max_plus1].shape)
                    # print('transit shape',throw_transit_probability[:,0:score_max_plus1].shape)     
                    child_probability[tokens_used:tokens_used+2,score_gained:score_gained+score_max_plus1] += throw_transit_probability[:,0:score_max_plus1]
                
                ## game can not bust or end when score_max = 60, i.e.,  prob_notbust = 1
                if (score_max < 60):
                    ## transit to the end of game
                    if (score_remain == fb.score_DB):
                        result_dict['finish'] += prob_bullscore[action_index, 1]*prob_this_state
                    elif (score_remain <= 40 and score_remain%2==0):
                        doublescore_index = (score_remain//2) - 1
                        result_dict['finish'] += prob_doublescore[action_index, doublescore_index]*prob_this_state
                    else:
                        pass

                    #transit to bust
                    result_dict['bust'][tokens_used+bool_token] += prob_bust_dic[score_max][action_index]*prob_this_state

                # if child_probability.sum()==0 and score_state > 2:#available_tokens > 0 and score_state == 59: 
                #     print('------------------------------------------------------------------------------------')
                #     print('score',score_state,'start tokens',available_tokens,'gained',score_gained,'tokens used',tokens_used)
                #     print('throw transit probs',throw_transit_probability)
                #     print('throw transit update vals',prob_normalscore[action_index][:max_throw_score_index])
                #     print('prob this state',prob_this_state)
                #     print('rt',rt)
                #     print('parent',parent_probability)
                #     print('child',child_probability)
                #     print('finish',result_dict['finish'])
                #     print('bust',result_dict['bust'])
                #     return -1

        parent_probability = child_probability
        #print(parent_probability)

    result_dict['score'] = parent_probability

    return result_dict
    

In [486]:
# Advance to next turn 
new_value_rt3 = 1 
# Don't use a token 
new_value_rt3 += np.dot(prob_turn_transit['score'][0,1:], optimal_value_rt3[tok,score_state-1:score_state-score_max_turn-1:-1])
# Use a token 
new_value_rt3 += np.dot(prob_turn_transit['score'][tokens_max_turn+1:0:-1,:][0], optimal_value_rt3[tok:tok-tokens_max_turn:-1,score_state:score_state-score_max_turn-1:-1][0])
new_value_rt3 += np.dot(prob_turn_transit['bust'][1:tokens_max_turn+1] , optimal_value_rt3[tok:tok-tokens_max_turn:-1,score_state])

# for t in range(1,prob_turn_transit['bust'].shape[0]): 
#     new_value_rt3 += np.dot(prob_turn_transit['score'][t,:], optimal_value_rt3[tok-t,score_state:score_state-score_max_turn-1:-1])
#     new_value_rt3 += prob_turn_transit['bust'][t] * optimal_value_rt3[tok-t,score_state]

new_value_rt3 

IndexError: index 2 is out of bounds for axis 0 with size 2

In [92]:
def init_score_state(score_state,state_len_vector,state_action,noturn_optimal_action_index,optimal_action_index_dic):     
    ## initialization 
    for rt in [1,2,3]:
        
        ## for rt=3: score_gained = 0
        ## for rt=2: score_gained = 0,1,...,min(s-2,60)
        ## for rt=1: score_gained = 0,1,...,min(s-2,120)
        this_throw_state_len = min(score_state-2, fb.maxhitscore*(3-rt)) + 1
        state_len_vector[rt] = this_throw_state_len
                
        ## initialize the starting policy: 
        ## use no_turn action in (s, i, u=0)
        ## use turn action (s-1, i, u-1) in (s, i, u!=0) if (s-1, i, u-1) is feasible state
        state_action[rt][0] = noturn_optimal_action_index[score_state]            
        for score_gained in range(1,this_throw_state_len):                
            if fb.state_feasible_array[rt, score_gained]:  ## if True
                if fb.state_feasible_array[rt, score_gained-1]:
                    state_action[rt][score_gained] = optimal_action_index_dic[score_state-1][rt][score_gained-1]
                else:                        
                    state_action[rt][score_gained] = noturn_optimal_action_index[score_state-score_gained]
            else:
                state_action[rt][score_gained] = fb.infeasible_marker

    return state_len_vector, state_action

In [87]:
def policy_evaluation(score_state,state_action,prob_grid_normalscore,prob_grid_doublescore,prob_grid_bullscore,prob_bust_dic,optimal_value_rt3,state_value_update): 

    rt = 3
    score_gained = 0
    score_max_turn = min(score_state-2, 3*fb.maxhitscore)

    prob_turn_transit = fep.solve_turn_transit_probability_fast(score_state, state_action, prob_grid_normalscore, prob_grid_doublescore, prob_grid_bullscore, prob_bust_dic)        
    prob_turn_zeroscore = prob_turn_transit['bust'] + prob_turn_transit['score'][0]
    new_value_rt3 = (1 + np.dot(prob_turn_transit['score'][1:], optimal_value_rt3[score_state-1:score_state-score_max_turn-1:-1])) / (1-prob_turn_zeroscore)
    state_value_update[rt][score_gained] = new_value_rt3
    optimal_value_rt3[score_state] = new_value_rt3

    return prob_turn_transit, prob_turn_zeroscore, new_value_rt3
    #print('evaluate rt3 value= {}'.format(new_value_rt3)

9

In [356]:
import copy
import helpers as h

def solve_dp_turn(aiming_grid, prob_grid_normalscore_nt, prob_grid_doublescore_nt, prob_grid_bullscore_nt, prob_grid_normalscore_t, prob_grid_doublescore_t, prob_grid_bullscore_t):
    """
    Solve the single player game with the turn feature.
    Args: 
        the action set and the hitting probability associated with the skill model
    
    Returns: 
        optimal values and the corresponding aiming locations for each state (s,i,u)
    """
    
    ## Initialize probability objects
    num_aiming_location, prob_normalscore_nt, prob_doublescore_dic_nt, prob_DB_nt, prob_bust_dic_nt, prob_notbust_dic_nt, prob_normalscore_t, prob_doublescore_dic_t, prob_DB_t, prob_bust_dic_t, prob_notbust_dic_t = init_probabilities( aiming_grid, prob_grid_normalscore_nt, prob_grid_doublescore_nt, prob_grid_bullscore_nt, prob_grid_normalscore_t, prob_grid_doublescore_t, prob_grid_bullscore_t)    
    prob_normalscore_tensor_nt = torch.from_numpy(prob_normalscore_nt)
    prob_normalscore_tensor_t = torch.from_numpy(prob_normalscore_t)

    prob_normalscore_tensor = torch.from_numpy(prob_normalscore_nt)
    prob_doublescore_dic = prob_doublescore_dic_nt
    prob_DB = prob_DB_nt
    prob_bust_dic = prob_bust_dic_nt
    prob_notbust_dic = prob_notbust_dic_nt
    
    iteration_round_limit = 20
    
    # Initialize Data Structures
    optimal_value_rt3, optimal_value_dic, optimal_action_index_dic, num_iteration_record, state_len_vector, state_value, state_action, action_diff, value_relerror, state_value_update, state_action_update = init_data_structures()
    
    ## use no_turn policy as the initial policy
    #[noturn_optimal_value, noturn_optimal_action_index] = solve_dp_noturn(aiming_grid, prob_grid_normalscore, prob_grid_doublescore, prob_grid_bullscore)
    # TODO: sync up tokens argument below with tokens of master function
    [noturn_optimal_value, noturn_optimal_action_index] = h.solve_dp_noturn_tokens(aiming_grid, prob_grid_normalscore_nt, prob_grid_normalscore_t, tokens = 0, prob_grid_doublescore=prob_grid_doublescore_nt, prob_grid_bullscore=prob_grid_bullscore_nt, prob_grid_doublescore_dic=None, prob_grid_doublescore_t=prob_grid_doublescore_t, prob_grid_bullscore_t=prob_grid_bullscore_t, prob_grid_doublescore_dic_t=None)
    # TODO: remove these once we start solving with tokens 
    noturn_optimal_value = noturn_optimal_value[0]
    noturn_optimal_action_index = noturn_optimal_action_index[0]

    t1 = time.time()
    for score_state in range(2, 502):
        #print('#### solve_dp_turn score_state={} ####'.format(score_state))    
        
        # Initialize data structures for this score state (i.e. for every number of remaining throws)
        state_len_vector, state_action = init_score_state(score_state,state_len_vector,state_action,noturn_optimal_action_index,optimal_action_index_dic)
    
        ## policy iteration
        for round_index in range(iteration_round_limit):
    
            ## policy evaluation for TURN 
            rt = 3
            score_gained = 0
            score_max_turn = min(score_state-2, 3*fb.maxhitscore)
            # TO DO 
            # pass token probabilities to solve_turn_transit_probability_fast
            # return prob_turn_transit with separate score values for token and non-token
            # "bust" and "finish" should probably be not impacted. 
            prob_turn_transit = fep.solve_turn_transit_probability_fast(score_state, state_action, prob_grid_normalscore, prob_grid_doublescore, prob_grid_bullscore, prob_bust_dic)        
            # This will be sum of overall bust + score_token[0] + score_notoken[0]
            prob_turn_zeroscore = prob_turn_transit['bust'] + prob_turn_transit['score'][0]
            
            # This can be staggered like the notoken case 
            # First we add 1 to indicate that we've incremented by a turn 
            # Then we take the dot product for non-token scores with the same token states
            # Then we take the dot product for the token scores with the next token states 

            new_value_rt3 = (1 + np.dot(prob_turn_transit['score'][1:], optimal_value_rt3[score_state-1:score_state-score_max_turn-1:-1])) / (1-prob_turn_zeroscore)
            #new_value_rt3 = 1 + np.dot(prob_turn_transit['score'][1:], optimal_value_rt3[score_state-1:score_state-score_max_turn-1:-1]) + (prob_turn_zeroscore)*optimal_value_rt3[score_state]

            # new_value_rt3 = 1 + np.dot 
            state_value_update[rt][score_gained] = new_value_rt3
            optimal_value_rt3[score_state] = new_value_rt3
            #print('evaluate rt3 value= {}'.format(new_value_rt3)     

            ## policy improvement
            for rt in [1,2,3]:            
                this_throw_state_len = state_len_vector[rt]
                
                ## state which can not bust.  score_state-score_gained>=62 
                state_notbust_len =  max(min(score_state-61, this_throw_state_len),0)
                if (state_notbust_len > 0):
                    if (rt==1 and round_index==0):
                        ## combine all non-bust states together 
                        state_notbust_update_index = state_notbust_len                    
                        next_state_value_array = np.zeros((61, state_notbust_len))                    
                        for score_gained in range(state_notbust_len):
                            ## skip infeasible state
                            if not fb.state_feasible_array[rt, score_gained]:
                                continue
                            score_remain = score_state - score_gained
                            score_max = 60 ## always 60 here
                            score_max_plus1 = score_max + 1
                            next_state_value_array[:,score_gained] = optimal_value_rt3[score_remain:score_remain-score_max_plus1:-1]
                    elif (rt==2 and (round_index==0 or score_state<182)):
                        ## combine all non-bust states together 
                        state_notbust_update_index = state_notbust_len
                        next_state_value_array = np.zeros((61, state_notbust_len))                    
                        for score_gained in range(state_notbust_len):
                            ## skip infeasible state
                            if not fb.state_feasible_array[rt, score_gained]:
                                continue
                            score_remain = score_state - score_gained
                            score_max = 60 ## always 60 here
                            score_max_plus1 = score_max + 1
                            next_state_value_array[:,score_gained] = state_value_update[rt-1][score_gained:score_gained+score_max_plus1]
                    else: ##(rt==1 and round_index>0) or (rt==2 and round_index>0 and score_state>=182) or (rt==3)
                        ## only update state of score_gained = 0
                        state_notbust_update_index = 1
                        next_state_value_array = np.zeros(61)
                        score_gained = 0
                        score_remain = score_state - score_gained
                        score_max = 60 ## always 60 here
                        score_max_plus1 = score_max + 1                    
                        ## make a copy
                        if (rt > 1):
                            next_state_value_array[:] = state_value_update[rt-1][score_gained:score_gained+score_max_plus1]
                        ## transit to next turn when rt=1
                        else:
                            next_state_value_array[:] = optimal_value_rt3[score_remain:score_remain-score_max_plus1:-1]
    
                    ## matrix product to compute all together
                    next_state_value_tensor = torch.from_numpy(next_state_value_array)
                    ## transit to next throw in the same turn when rt=3,2
                    if (rt > 1):                    
                        num_turns_tensor = prob_normalscore_tensor.matmul(next_state_value_tensor)
                    ## transit to next turn when rt=1
                    else:
                        num_turns_tensor = 1 + prob_normalscore_tensor.matmul(next_state_value_tensor)
    
                    ## searching
                    temp1 = num_turns_tensor.min(axis=0)                
                    state_action_update[rt][0:state_notbust_update_index] = temp1.indices.numpy()
                    state_value_update[rt][0:state_notbust_update_index] =  temp1.values.numpy()                
                
                ## state which possibly bust.  score_state-score_gained<62 
                if (state_notbust_len < this_throw_state_len):
                    ## combine all bust states together 
                    state_bust_len = this_throw_state_len - state_notbust_len
                    next_state_value_array = np.zeros((61, state_bust_len))
                    for score_gained in range(state_notbust_len, this_throw_state_len):
                        ## skip infeasible state
                        if not fb.state_feasible_array[rt, score_gained]:
                            continue
                        score_remain = score_state - score_gained
                        #score_max = min(score_remain-2, 60)
                        score_max = score_remain-2 ## less than 60 here
                        score_max_plus1 = score_max + 1
                        score_gained_index = score_gained - state_notbust_len ## index off set
                        if (rt > 1):
                            next_state_value_array[0:score_max_plus1,score_gained_index] = state_value_update[rt-1][score_gained:score_gained+score_max_plus1]
                        ## transit to next turn when rt=1
                        else:
                            next_state_value_array[0:score_max_plus1,score_gained_index] = optimal_value_rt3[score_remain:score_remain-score_max_plus1:-1]
                    
                    next_state_value_tensor = torch.from_numpy(next_state_value_array)
                    ## transit to next throw in the same turn when rt=3,2
                    if (rt > 1):                    
                        num_turns_tensor = prob_normalscore_tensor.matmul(next_state_value_tensor)
                    ## transit to next turn when rt=1
                    else:
                        num_turns_tensor = 1 + prob_normalscore_tensor.matmul(next_state_value_tensor)                                                               
    
                    ## consider bust/finishing for each bust state seperately 
                    num_turns_array = num_turns_tensor.numpy()                
                    for score_gained in range(state_notbust_len, this_throw_state_len):
                        ## skip infeasible state
                        if not fb.state_feasible_array[rt, score_gained]:
                            continue
                        score_remain = score_state - score_gained
                        #score_max = min(score_remain-2, 60)
                        score_max = score_remain-2 ## less than 60 here
                        score_max_plus1 = score_max + 1
                        score_gained_index = score_gained - state_notbust_len
    
                        ## transit to the end of game
                        if (rt > 1):
                            if (score_remain == fb.score_DB):                        
                                num_turns_array[:,score_gained_index] += prob_DB
                            elif (score_remain <= 40 and score_remain%2==0):
                                num_turns_array[:,score_gained_index] += prob_doublescore_dic[score_remain]
                            else:
                                pass
    
                        ## transit to bust
                        if (rt==3):
                            num_turns_array[:,score_gained_index] += prob_bust_dic[score_max]
                            ## solve an equation other than using the policy evaluation value (s,i=3,u=0)
                            num_turns_array[:,score_gained_index] = num_turns_array[:,score_gained_index] / prob_notbust_dic[score_max] 
                        elif (rt==2):
                            num_turns_array[:,score_gained_index] += prob_bust_dic[score_max]*(1+new_value_rt3)
                        else:
                            num_turns_array[:,score_gained_index] += prob_bust_dic[score_max]*(new_value_rt3)  ## 1 turn is already counted before
    
                    ## searching
                    temp1 = num_turns_tensor.min(axis=0)
                    state_action_update[rt][state_notbust_len:this_throw_state_len] = temp1.indices.numpy()
                    state_value_update[rt][state_notbust_len:this_throw_state_len] =  temp1.values.numpy()                
    
                #### finish rt=1,2,3. check improvement
                action_diff[rt][:] = np.abs(state_action_update[rt] - state_action[rt])                                
                value_relerror[rt] = np.abs((state_value_update[rt] - state_value[rt])/state_value_update[rt]).max()
                state_action[rt][:] = state_action_update[rt][:]
                state_value[rt][:] = state_value_update[rt][:]
    
            max_action_diff = max([action_diff[1].max(), action_diff[2].max(), action_diff[3].max()])
            max_value_relerror = value_relerror.max()            
            
            if (max_action_diff < 1):
            #if max_value_relerror < iteration_relerror_limit:
                num_iteration_record[score_state] = round_index + 1
                break
    
        for rt in [1,2,3]:
            state_value_update[rt][fb.state_infeasible[rt]] = fb.largenumber
            state_action_update[rt][fb.state_infeasible[rt]] = fb.infeasible_marker
            
        optimal_action_index_dic[score_state] = ft.copy_numberarray_container(state_action_update)
        optimal_value_dic[score_state] = ft.copy_numberarray_container(state_value_update, new_dtype=fb.result_float_dytpe)
        optimal_value_rt3[score_state] = state_value[3][0]
        ## done:V(s,i=3/2/1,u)
    
    ##
    prob_scorestate_transit = {}    
    prob_scorestate_transit =  fep.solve_policy_transit_probability(optimal_action_index_dic, prob_grid_normalscore, prob_grid_doublescore, prob_grid_bullscore)
    t2 = time.time()
    print('solve dp_turn_policyiter in {} seconds'.format(t2-t1))

    print(optimal_value_rt3)
    result_dic = {'optimal_value_dic':optimal_value_dic, 'optimal_action_index_dic':optimal_action_index_dic, 'optimal_value_rt3':optimal_value_rt3, 'prob_scorestate_transit':prob_scorestate_transit}

    return result_dic 


In [350]:
[noturn_optimal_value, noturn_optimal_action_index] = solve_dp_noturn(aiming_grid, prob_grid_normalscore, prob_grid_doublescore, prob_grid_bullscore)
#[noturn_optimal_value, noturn_optimal_action_index] = h.solve_dp_noturn_tokens(aiming_grid, prob_grid_normalscore_nt, prob_grid_normalscore_t, tokens = 0, prob_grid_doublescore=prob_grid_doublescore_nt, prob_grid_bullscore=prob_grid_bullscore_nt, prob_grid_doublescore_dic=None, prob_grid_doublescore_t=prob_grid_doublescore_t, prob_grid_bullscore_t=prob_grid_bullscore_t, prob_grid_doublescore_dic_t=None)
noturn_optimal_value

/var/folders/jj/jcq76kp53_zchkx8b5cvhdsh0000gn/T/ipykernel_8165/78220377.py:59: RuntimeWarning: divide by zero encountered in divide
  num_tothrow = num_tothrow / prob_otherstate
/var/folders/jj/jcq76kp53_zchkx8b5cvhdsh0000gn/T/ipykernel_8165/78220377.py:59: RuntimeWarning: overflow encountered in divide
  num_tothrow = num_tothrow / prob_otherstate


array([ 0.    ,  0.    ,  2.0893, ..., 14.0691, 14.1144, 14.1292])

In [303]:
[aiming_grid, prob_grid_normalscore_nt, prob_grid_singlescore_nt, prob_grid_doublescore_nt, prob_grid_triplescore_nt, prob_grid_bullscore_nt] = fgag.load_aiming_grid(name_pa, data_parameter_dir=data_parameter_dir, grid_version='custom_no_tokens')
[aiming_grid, prob_grid_normalscore_t, prob_grid_singlescore_t, prob_grid_doublescore_t, prob_grid_triplescore_t, prob_grid_bullscore_t] = fgag.load_aiming_grid(name_pa, data_parameter_dir=data_parameter_dir, grid_version='custom_tokens')

result_dic = solve_dp_turn(aiming_grid, prob_grid_normalscore_nt, prob_grid_doublescore_nt, prob_grid_bullscore_nt)#, prob_grid_normalscore_t, prob_grid_doublescore_t, prob_grid_bullscore_t)


load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_no_tokens/player10_gaussin_prob_grid_custom_no_tokens.pkl
load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_tokens/player10_gaussin_prob_grid_custom_tokens.pkl


/var/folders/jj/jcq76kp53_zchkx8b5cvhdsh0000gn/T/ipykernel_964/78220377.py:59: RuntimeWarning: divide by zero encountered in divide
  num_tothrow = num_tothrow / prob_otherstate
/var/folders/jj/jcq76kp53_zchkx8b5cvhdsh0000gn/T/ipykernel_964/78220377.py:59: RuntimeWarning: overflow encountered in divide
  num_tothrow = num_tothrow / prob_otherstate
/var/folders/jj/jcq76kp53_zchkx8b5cvhdsh0000gn/T/ipykernel_964/3007519047.py:223: RuntimeWarning: divide by zero encountered in divide
  num_turns_array[:,score_gained_index] = num_turns_array[:,score_gained_index] / prob_notbust_dic[score_max]
/var/folders/jj/jcq76kp53_zchkx8b5cvhdsh0000gn/T/ipykernel_964/3007519047.py:223: RuntimeWarning: overflow encountered in divide
  num_turns_array[:,score_gained_index] = num_turns_array[:,score_gained_index] / prob_notbust_dic[score_max]
/var/folders/jj/jcq76kp53_zchkx8b5cvhdsh0000gn/T/ipykernel_964/3007519047.py:236: RuntimeWarning: divide by zero encountered in divide
  value_relerror[rt] = np.abs((

solve prob_policy_transit in 0.2481679916381836 seconds
solve dp_turn_policyiter in 1.8161218166351318 seconds
[0.     0.     1.4381 ... 5.0818 5.0845 5.0965]


In [1209]:
## aiming_grid
num_aiming_location = aiming_grid.shape[0]
prob_normalscore = prob_grid_normalscore_nt
prob_doublescore_dic = {}
for doublescore_index in range(20):
    doublescore = 2*(doublescore_index+1)
    prob_doublescore_dic[doublescore] = np.array(prob_grid_doublescore_nt[:,doublescore_index])
prob_DB = np.array(prob_grid_bullscore_nt[:,1])

## the probability of not bust for each action given score_max=i (score_remain=i+2)
prob_bust_dic = {}
prob_notbust_dic = {}
for score_max in range(60):    
    ## transit to next throw or turn
    prob_notbust = prob_grid_normalscore_nt[:,0:score_max+1].sum(axis=1)
    ## transit to the end of game
    score_remain = score_max + 2
    if (score_remain == fb.score_DB):
        prob_notbust += prob_DB
    elif (score_remain <= 40 and score_remain%2==0):
        prob_notbust += prob_doublescore_dic[score_remain]
    ##
    prob_notbust = np.minimum(np.maximum(prob_notbust, 0),1)
    prob_notbust_dic[score_max] = prob_notbust
    prob_bust_dic[score_max] = 1 - prob_notbust_dic[score_max]

prob_normalscore_tensor = torch.from_numpy(prob_normalscore)

iteration_round_limit = 20
iteration_relerror_limit = 10**-9

#### state space example of (SB=25 DB=50) ####
## rt: the number of remaining throws in a turn
## state_infeasible_rt2 = [23, 29, 31, 35, 37, 41, 43, 44, 46, 47, 49, 52, 53, 55, 56, 58, 59]
## state_infeasible_rt1 = [103, 106, 109, 112, 113, 115, 116, 118, 119]    
    
optimal_value_rt3 = np.zeros(502) #vector: optimal value for the beginning state of each turn (rt=3)
optimal_value_dic = {} ## first key: score=0,2,...,501, second key: remaining throws=3,2,1
optimal_action_index_dic = {}
num_iteration_record = np.zeros(502, dtype=np.int32)

state_len_vector = np.zeros(4, dtype=np.int32)
state_value  = [None]  ## optimal value (expected # of turns to finish the game) for each state in the current playing turn
state_action = [None]  ## aimming locations for for each state in the current playing turn
action_diff  = [None]
value_relerror = np.zeros(4)
for rt in [1,2,3]:
    ## for rt=3: possible score_gained = 0
    ## for rt=2: possible score_gained = 0,1,...,60
    ## for rt=1: possible score_gained = 0,1,...,120
    this_throw_state_len = fb.maxhitscore*(3-rt) + 1
    state_value.append(np.ones(this_throw_state_len)*fb.largenumber)
    state_action.append(np.ones(this_throw_state_len, np.int32)*fb.infeasible_marker)
    action_diff.append(np.ones(this_throw_state_len))
state_value_update = ft.copy_numberarray_container(state_value)
state_action_update = ft.copy_numberarray_container(state_action)

## use no_turn policy as the initial policy
[noturn_optimal_value, noturn_optimal_action_index] = solve_dp_noturn(aiming_grid, prob_grid_normalscore_nt, prob_grid_doublescore_nt, prob_grid_bullscore_nt)

t1 = time.time()
for score_state in range(2, 502):
    #print('#### solve_dp_turn score_state={} ####'.format(score_state))    
    
    ## initialization 
    for rt in [1,2,3]:
        ## for rt=3: score_gained = 0
        ## for rt=2: score_gained = 0,1,...,min(s-2,60)
        ## for rt=1: score_gained = 0,1,...,min(s-2,120)
        this_throw_state_len = min(score_state-2, fb.maxhitscore*(3-rt)) + 1
        state_len_vector[rt] = this_throw_state_len
                
        ## initialize the starting policy: 
        ## use no_turn action in (s, i, u=0)
        ## use turn action (s-1, i, u-1) in (s, i, u!=0) if (s-1, i, u-1) is feasible state
        state_action[rt][0] = noturn_optimal_action_index[score_state]            
        for score_gained in range(1,this_throw_state_len):                
            if fb.state_feasible_array[rt, score_gained]:  ## if True
                if fb.state_feasible_array[rt, score_gained-1]:
                    state_action[rt][score_gained] = optimal_action_index_dic[score_state-1][rt][score_gained-1]
                else:                        
                    state_action[rt][score_gained] = noturn_optimal_action_index[score_state-score_gained]
            else:
                state_action[rt][score_gained] = fb.infeasible_marker

    ## policy iteration
    for round_index in range(iteration_round_limit):

        ## policy evaluation
        rt = 3
        score_gained = 0
        score_max_turn = min(score_state-2, 3*fb.maxhitscore)
        prob_turn_transit = fep.solve_turn_transit_probability_fast(score_state, state_action, prob_grid_normalscore_nt, prob_grid_doublescore_nt, prob_grid_bullscore_nt, prob_bust_dic)        
        prob_turn_zeroscore = prob_turn_transit['bust'] + prob_turn_transit['score'][0]
        new_value_rt3 = (1 + np.dot(prob_turn_transit['score'][1:], optimal_value_rt3[score_state-1:score_state-score_max_turn-1:-1])) / (1-prob_turn_zeroscore)
        state_value_update[rt][score_gained] = new_value_rt3
        optimal_value_rt3[score_state] = new_value_rt3
        #print('evaluate rt3 value= {}'.format(new_value_rt3)

        # ## policy improvement
        for rt in [1,2,3]:            
            this_throw_state_len = state_len_vector[rt]
            
            ## state which can not bust.  score_state-score_gained>=62 
            state_notbust_len =  max(min(score_state-61, this_throw_state_len),0)
            if (state_notbust_len > 0):
                if (rt==1 and round_index==0):
                    ## combine all non-bust states together 
                    state_notbust_update_index = state_notbust_len                    
                    next_state_value_array = np.zeros((61, state_notbust_len))                    
                    for score_gained in range(state_notbust_len):
                        ## skip infeasible state
                        if not fb.state_feasible_array[rt, score_gained]:
                            continue
                        score_remain = score_state - score_gained
                        score_max = 60 ## always 60 here
                        score_max_plus1 = score_max + 1
                        next_state_value_array[:,score_gained] = optimal_value_rt3[score_remain:score_remain-score_max_plus1:-1]
                elif (rt==2 and (round_index==0 or score_state<182)):
                    ## combine all non-bust states together 
                    state_notbust_update_index = state_notbust_len
                    next_state_value_array = np.zeros((61, state_notbust_len))                    
                    for score_gained in range(state_notbust_len):
                        ## skip infeasible state
                        if not fb.state_feasible_array[rt, score_gained]:
                            continue
                        score_remain = score_state - score_gained
                        score_max = 60 ## always 60 here
                        score_max_plus1 = score_max + 1
                        next_state_value_array[:,score_gained] = state_value_update[rt-1][score_gained:score_gained+score_max_plus1]
                else: ##(rt==1 and round_index>0) or (rt==2 and round_index>0 and score_state>=182) or (rt==3)
                    ## only update state of score_gained = 0
                    state_notbust_update_index = 1
                    next_state_value_array = np.zeros(61)
                    score_gained = 0
                    score_remain = score_state - score_gained
                    score_max = 60 ## always 60 here
                    score_max_plus1 = score_max + 1                    
                    ## make a copy
                    if (rt > 1):
                        next_state_value_array[:] = state_value_update[rt-1][score_gained:score_gained+score_max_plus1]
                    ## transit to next turn when rt=1
                    else:
                        next_state_value_array[:] = optimal_value_rt3[score_remain:score_remain-score_max_plus1:-1]

                ## matrix product to compute all together
                next_state_value_tensor = torch.from_numpy(next_state_value_array)
                ## transit to next throw in the same turn when rt=3,2
                if (rt > 1):                    
                    num_turns_tensor = prob_normalscore_tensor.matmul(next_state_value_tensor)
                ## transit to next turn when rt=1
                else:
                    num_turns_tensor = 1 + prob_normalscore_tensor.matmul(next_state_value_tensor)

                ## searching
                temp1 = num_turns_tensor.min(axis=0)                
                state_action_update[rt][0:state_notbust_update_index] = temp1.indices.numpy()
                state_value_update[rt][0:state_notbust_update_index] =  temp1.values.numpy()                
            
            ## state which possibly bust.  score_state-score_gained<62 
            if (state_notbust_len < this_throw_state_len):
                ## combine all bust states together 
                state_bust_len = this_throw_state_len - state_notbust_len
                next_state_value_array = np.zeros((61, state_bust_len))
                for score_gained in range(state_notbust_len, this_throw_state_len):
                    ## skip infeasible state
                    if not fb.state_feasible_array[rt, score_gained]:
                        continue
                    score_remain = score_state - score_gained
                    #score_max = min(score_remain-2, 60)
                    score_max = score_remain-2 ## less than 60 here
                    score_max_plus1 = score_max + 1
                    score_gained_index = score_gained - state_notbust_len ## index off set
                    if (rt > 1):
                        next_state_value_array[0:score_max_plus1,score_gained_index] = state_value_update[rt-1][score_gained:score_gained+score_max_plus1]
                    ## transit to next turn when rt=1
                    else:
                        next_state_value_array[0:score_max_plus1,score_gained_index] = optimal_value_rt3[score_remain:score_remain-score_max_plus1:-1]
                
                next_state_value_tensor = torch.from_numpy(next_state_value_array)
                ## transit to next throw in the same turn when rt=3,2
                if (rt > 1):                    
                    num_turns_tensor = prob_normalscore_tensor.matmul(next_state_value_tensor)
                ## transit to next turn when rt=1
                else:
                    num_turns_tensor = 1 + prob_normalscore_tensor.matmul(next_state_value_tensor)                                                               

                ## consider bust/finishing for each bust state seperately 
                num_turns_array = num_turns_tensor.numpy()                
                for score_gained in range(state_notbust_len, this_throw_state_len):
                    ## skip infeasible state
                    if not fb.state_feasible_array[rt, score_gained]:
                        continue
                    score_remain = score_state - score_gained
                    #score_max = min(score_remain-2, 60)
                    score_max = score_remain-2 ## less than 60 here
                    score_max_plus1 = score_max + 1
                    score_gained_index = score_gained - state_notbust_len

                    ## transit to the end of game
                    if (rt > 1):
                        if (score_remain == fb.score_DB):                        
                            num_turns_array[:,score_gained_index] += prob_DB
                        elif (score_remain <= 40 and score_remain%2==0):
                            num_turns_array[:,score_gained_index] += prob_doublescore_dic[score_remain]
                        else:
                            pass

                    ## transit to bust
                    if (rt==3):
                        num_turns_array[:,score_gained_index] += prob_bust_dic[score_max]
                        ## solve an equation other than using the policy evaluation value (s,i=3,u=0)
                        num_turns_array[:,score_gained_index] = num_turns_array[:,score_gained_index] / prob_notbust_dic[score_max] 
                    elif (rt==2):
                        num_turns_array[:,score_gained_index] += prob_bust_dic[score_max]*(1+new_value_rt3)
                    else:
                        num_turns_array[:,score_gained_index] += prob_bust_dic[score_max]*(new_value_rt3)  ## 1 turn is already counted before

                ## searching
                temp1 = num_turns_tensor.min(axis=0)
                state_action_update[rt][state_notbust_len:this_throw_state_len] = temp1.indices.numpy()
                state_value_update[rt][state_notbust_len:this_throw_state_len] =  temp1.values.numpy()                

            #### finish rt=1,2,3. check improvement
            action_diff[rt][:] = np.abs(state_action_update[rt] - state_action[rt])                                
            value_relerror[rt] = np.abs((state_value_update[rt] - state_value[rt])/state_value_update[rt]).max()
            state_action[rt][:] = state_action_update[rt][:]
            state_value[rt][:] = state_value_update[rt][:]

        max_action_diff = max([action_diff[1].max(), action_diff[2].max(), action_diff[3].max()])
        max_value_relerror = value_relerror.max()            
        
        if (max_action_diff < 1):
        #if max_value_relerror < iteration_relerror_limit:
            num_iteration_record[score_state] = round_index + 1
            break

    for rt in [1,2,3]:
        state_value_update[rt][fb.state_infeasible[rt]] = fb.largenumber
        state_action_update[rt][fb.state_infeasible[rt]] = fb.infeasible_marker
    optimal_action_index_dic[score_state] = ft.copy_numberarray_container(state_action_update)
    optimal_value_dic[score_state] = ft.copy_numberarray_container(state_value_update, new_dtype=fb.result_float_dytpe)
    optimal_value_rt3[score_state] = state_value[3][0]
    ## done:V(s,i=3/2/1,u)

##
prob_scorestate_transit = {}    
prob_scorestate_transit =  fep.solve_policy_transit_probability(optimal_action_index_dic, prob_grid_normalscore_nt, prob_grid_doublescore_nt, prob_grid_bullscore_nt)
t2 = time.time()
print('solve dp_turn_policyiter in {} seconds'.format(t2-t1))

print(optimal_value_rt3)
result_dic = {'optimal_value_dic':optimal_value_dic, 'optimal_action_index_dic':optimal_action_index_dic, 'optimal_value_rt3':optimal_value_rt3, 'prob_scorestate_transit':prob_scorestate_transit}


/var/folders/jj/jcq76kp53_zchkx8b5cvhdsh0000gn/T/ipykernel_964/78220377.py:59: RuntimeWarning: divide by zero encountered in divide
  num_tothrow = num_tothrow / prob_otherstate
/var/folders/jj/jcq76kp53_zchkx8b5cvhdsh0000gn/T/ipykernel_964/78220377.py:59: RuntimeWarning: overflow encountered in divide
  num_tothrow = num_tothrow / prob_otherstate
/var/folders/jj/jcq76kp53_zchkx8b5cvhdsh0000gn/T/ipykernel_964/228615347.py:213: RuntimeWarning: divide by zero encountered in divide
  num_turns_array[:,score_gained_index] = num_turns_array[:,score_gained_index] / prob_notbust_dic[score_max]
/var/folders/jj/jcq76kp53_zchkx8b5cvhdsh0000gn/T/ipykernel_964/228615347.py:213: RuntimeWarning: overflow encountered in divide
  num_turns_array[:,score_gained_index] = num_turns_array[:,score_gained_index] / prob_notbust_dic[score_max]
/var/folders/jj/jcq76kp53_zchkx8b5cvhdsh0000gn/T/ipykernel_964/228615347.py:226: RuntimeWarning: divide by zero encountered in divide
  value_relerror[rt] = np.abs((sta

solve prob_policy_transit in 0.2382669448852539 seconds
solve dp_turn_policyiter in 1.9089641571044922 seconds
[0.     0.     1.8856 ... 5.5944 5.5758 5.6116]
